In [1]:
import pandas as pd
import numpy as np
from dateutil.parser import parse

#import and read data
samples = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/AmostrasAngolaTerrario.xlsx")
analyses = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Horizontes Analises.xlsx")
morphology = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Horizontes_Morfologia.xlsx")
profile_loc = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_local.xlsx")
soil_profile = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_solo.xlsx")
elemental_analyses = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Data XRF Angola_inicial.xlsx")
#soil_type = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_solo.xlsx")

# Samples table

In [3]:
import pandas as pd

# Load original dataset
samples = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/AmostrasAngolaTerrario.xlsx")

# This will show all rows that are duplicates (keep=False shows all duplicates, not just subsequent ones)
duplicates1 = samples[samples.duplicated('Registo',keep=False)]
print(duplicates1)

Empty DataFrame
Columns: [Registo, Nº Campo, Ano, Perfil, Campanha, Colónia_Pais, Distrito, AmostraCrivada, Província, AmostraNaoCrivada, Prateleira, Sala, Obs]
Index: []


In [4]:
# Rename columns for clarity
samples.rename(columns={
    'Registo': 'sample_id',
    'Nº Campo': 'site_info_id',
    'Ano': 'year',
    'Perfil': 'profile',
    'Campanha': 'campaign',
    'Colónia_Pais': 'country',
    'Distrito': 'district',
    'AmostraCrivada': 'sample_sifted',
    'AmostraNaoCrivada': 'sample_not_sifted',
    'Prateleira': 'shelf',
    'Sala': 'room'
}, inplace=True)

# Drop unused columns
samples_cleaning = samples.drop(columns=[
    'campaign', 'country', 'Província', 'sample_not_sifted', 'sample_sifted', 'Obs'
], errors='ignore')

# # Add a new Primary Key ID column starting from 1
# samples_cleaning.insert(0, 'sample_id', range(1, len(samples_cleaning) + 1))

# Add empty FK columns
samples_cleaning['lab_info_id'] = pd.NA
samples_cleaning['horizon_id'] = pd.NA
samples_cleaning['profile_record_id'] = pd.NA

In [5]:
samples_cleaning.head()

,sample_id,site_info_id,year,profile,district,shelf,room,lab_info_id,horizon_id,profile_record_id
0,630,172,1946.0,139,Huambo,1,22,<NA>,<NA>,<NA>
1,631,173,1946.0,139,Huambo,1,22,<NA>,<NA>,<NA>
2,632,174,1946.0,139,Huambo,1,22,<NA>,<NA>,<NA>
3,633,175,1946.0,139,Huambo,1,22,<NA>,<NA>,<NA>
4,687,1034,1946.0,208,Huambo,1,22,<NA>,<NA>,<NA>


In [6]:
samples_cleaning.dtypes

sample_id              int64
site_info_id          object
year                 float64
profile               object
district              object
shelf                 object
room                  object
lab_info_id           object
horizon_id            object
profile_record_id     object
dtype: object

In [7]:
# Ensure consistent types and formatting
samples_cleaning['profile'] = samples_cleaning['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]
samples_cleaning['shelf'] = samples_cleaning['shelf'].astype("string")

#formatting site_info_id
samples_cleaning['site_info_id'] = samples_cleaning['site_info_id'].astype(str).str.replace('/', '_').str.strip().str[:20]

# Convert sample_id to string (no truncation unless necessary)
samples_cleaning['sample_id'] = samples_cleaning['sample_id'].astype(str).str.strip()

# Check for duplicates AFTER conversion
duplicate_ids = samples_cleaning[samples_cleaning.duplicated('sample_id', keep=False)]
print("🔍 Duplicate sample_id values:")
print(duplicate_ids[['sample_id']])

samples_cleaning1 = samples_cleaning



🔍 Duplicate sample_id values:
Empty DataFrame
Columns: [sample_id]
Index: []


Checking which profiles match

In [9]:
# Reorder to match DB schema
samples_check = samples_cleaning1[[
    'sample_id',
    'site_info_id',
    'profile_record_id',
    'profile',
    'horizon_id',
    'shelf',
    'room',
    'year'
]]

samples_check.head()

,sample_id,site_info_id,profile_record_id,profile,horizon_id,shelf,room,year
0,630,172,<NA>,139,<NA>,1,22,1946.0
1,631,173,<NA>,139,<NA>,1,22,1946.0
2,632,174,<NA>,139,<NA>,1,22,1946.0
3,633,175,<NA>,139,<NA>,1,22,1946.0
4,687,1034,<NA>,208,<NA>,1,22,1946.0


In [10]:
samples_check.dtypes

sample_id                    object
site_info_id                 object
profile_record_id            object
profile                      object
horizon_id                   object
shelf                string[python]
room                         object
year                        float64
dtype: object

In [10]:
#samples_check.to_csv("/Users/inesschwartz/Desktop/samples_check.csv", index=False)

# Lab_Info  table

In [11]:
analyses.columns

Index(['Amostra', 'ID', 'Morfo_id', 'Analise-id', 'PERFIL', 'GR', 'COD_PROV',
       'NA', 'LS', 'LI', 'EG', 'AG', 'AF', 'L', 'Argila', 'Eq_Hum', 'atm_1/3',
       'atm_15', 'CACO3', 'Gesso', 'Fe livre', 'CO', 'N total', 'C/N', 'MO',
       'P205 total', 'pH (H2O)', 'pH(KCL)', 'Ca++', 'Mg++', 'K+', 'Na+',
       'Soma de bases', 'CTC', 'V', 'Cloretos', 'Sulfatos', 'Condutividade',
       'Sódio solúvel', 'EqMol (SiO2)', 'EqMol(Al2O3)', 'EqMol(Fe2O3)',
       'SiO2/Al2O3', 'SiO2/Fe2O3', 'SiO2/R2O3', 'Fe2O3/Al2O3', 'FE2O3_TARG',
       'FE2O3_LARG', 'CEC_ARG', 'Min_<0,002', 'Min_0,05-0,02', 'Min_0,2-0,05',
       'Min_2-0,2', 'Confirmar'],
      dtype='object')

In [12]:
#change column names
analyses.rename(columns={
    'Amostra': 'sample_id',
    'Morfo_id': 'horizon_id',
    'Analise-id': 'analysis_id',
    'PERFIL': 'profile', #change to reference PK profile_record_id
    'LS': 'upper_limit',
    'LI': 'lower_limit',
    'EG': 'EG',
    'AG': 'thick_clay',
    'AF': 'fine_clay',
    'Argila': 'clay',
    'L': 'silt',
    'Gesso': 'gypsum',
    'Fe livre': 'free_iron',
    'CO':'organic_carbon',
    'N total': 'total_N',
    'MO': 'OM',
    'Soma de bases': 'exchangable_bases_sum',
    'CTC': 'CEC',
    'Cloretos':'chlorides',
    'Sulfatos':'sulfates',
    'Condutividade':'conductivity',
    'Sódio solúvel':'soluble_sodium',
    'P205 total': 'P205',
    'pH (H2O)':'pH_H2O',
    'pH(KCL)':'pH_KCL',
    'MO': 'organic_material',
    'atm_1/3': 'atm_1_3',
    'Ca++': 'Ca',
    'Mg++': 'Mg',
    'Na+': 'Na',
    'K+': 'K',
    'Min_<0,002': 'Min_lt_0002',
    'Min_0,05-0,02': 'Min_005_002',
    'Min_0,2-0,05': 'Min_02_005',
    'Min_2-0,2': 'Min_2_02',
}, inplace=True)

# Drop columns not needed
analyses_drop = analyses.drop([
    'EqMol (SiO2)', 'EqMol(Al2O3)', 'EqMol(Fe2O3)', 'SiO2/Al2O3', 
    'SiO2/Fe2O3', 'SiO2/R2O3', 'Fe2O3/Al2O3', 'FE2O3_TARG', 
    'FE2O3_LARG', 'CEC_ARG', 'GR', 'COD_PROV'
], axis=1)

In [13]:
# Add a new Primary Key ID column starting from 1
#drop lab_sample_id if already exist error comes up
# Check if 'lab_sample_id' exists and drop it
if 'lab_sample_id' in analyses_drop.columns:
    analyses_drop = analyses_drop.drop(columns=['lab_sample_id'])

# Insert a new column, e.g., 'lab_sample_id' as a primary key starting from 1
analyses_drop.insert(0, 'lab_sample_id', range(1, len(analyses_drop) + 1))


#add minerology_id column
analyses_drop['minerology_id'] = pd.NA
#add soil_biology_id column (just in case)
analyses_drop['soil_biology_id'] = pd.NA


#Define the desired final column order
final_columns = [
    'lab_sample_id',
    'ID',
    'analysis_id',
    'horizon_id',
    'sample_id',
    'profile',
    'minerology_id',
    'soil_biology_id',
    'EG',
    'thick_clay',
    'fine_clay',
    'silt',
    'clay',
    'Eq_Hum',
    'atm_1_3',
    'atm_15',
    'CACO3',
    'gypsum',
    'free_iron',
    'organic_carbon',
    'total_N',
    'P205',
    'organic_material',
    'pH_H2O',
    'pH_KCL',
    'Ca',
    'Mg',
    'Na',
    'K',
    'exchangable_bases_sum',
    'CEC',
    'V',
    'conductivity',
    'soluble_sodium',
    'Min_lt_0002',
    'Min_005_002',
    'Min_02_005',
    'Min_2_02',
]

analyses1 = analyses_drop[final_columns]

In [14]:
#address nulls 
# Show count of NaN values in each column
nan_counts = analyses1.isna().sum()

# Filter to show only columns with at least one NaN
nan_columns = nan_counts[nan_counts > 0]

# Print them
print("🔍 Columns with NaN values:")
print(nan_columns)


🔍 Columns with NaN values:
sample_id                  19
minerology_id            7847
soil_biology_id          7847
EG                       4910
thick_clay                448
fine_clay                 390
silt                      430
clay                      420
Eq_Hum                   2297
atm_1_3                  7469
atm_15                   5689
CACO3                    7334
gypsum                   7815
free_iron                2792
organic_carbon           2395
total_N                  6259
P205                     5464
organic_material         2413
pH_H2O                    684
pH_KCL                   1749
Ca                       3018
Mg                       3025
Na                       3024
K                        2995
exchangable_bases_sum    6740
CEC                      2423
V                        2476
conductivity             7201
soluble_sodium           1270
Min_lt_0002              7517
Min_005_002              7719
Min_02_005               7723
Min_2_02     

In [15]:
#formatting and checking for duplicates
# Make a safe copy
analyses1 = analyses_drop[final_columns].copy()

# Convert sample_id to numeric, coercing errors to NaN
analyses1['sample_id'] = pd.to_numeric(analyses1['sample_id'], errors='coerce')

# Drop rows with NaN in sample_id to avoid int conversion error
analyses1 = analyses1.dropna(subset=['sample_id'])

# Convert to int, then to str
analyses1['sample_id'] = analyses1['sample_id'].astype(int).astype(str)

# Clean the profile column
analyses1['profile'] = (
    analyses1['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str[:20]
)

# Alias to another variable
analyses2 = analyses1

In [16]:
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = analyses2['sample_id'][analyses2['sample_id'].duplicated()].unique()
print(duplicated_values)

['10337' '11608' '3497' '3498' '2923' '2924' '2925' '2926' '2927' '2928'
 '2929' '2930' '16355' '16356' '419' '420' '15682' '13817' '75' '13965'
 '3418' '9230' '8639' '3529']


In [17]:
# 1. Ensure sample_id and analysis_id are strings
analyses2['sample_id'] = analyses2['sample_id'].astype(str).str.strip()
analyses2['analysis_id'] = analyses2['analysis_id'].astype(str).str.strip()

# 2. Correct duplicates for sample_id '11608'
mask_11608 = analyses2['sample_id'] == '11608'
analyses2.loc[mask_11608, 'sample_id'] = [
    f"11608_{i+1}" for i in range(mask_11608.sum())
]

# 3. Remove duplicate for sample_id '3497' (keep first occurrence only)
duplicates_3497 = analyses2[analyses2['sample_id'] == '3497']
if len(duplicates_3497) > 1:
    indices_to_drop = duplicates_3497.index[1:]
    analyses2 = analyses2.drop(indices_to_drop)

# 4. Correct sample_id → analysis_id assignments for specific sample_ids
id_map = {
    'UZ_24c/60_1_1': '8639',
    'UZ_24c/60_2_1': '8640',
    'UZ_24c/60_3_1': '8641',
    'UZ_24c/60_4_1': '8642',
    'UZ_24c/60_5_1': '8643'
}

# Apply mapping to update analysis_id based on sample_id
analyses2['analysis_id'] = analyses2.apply(
    lambda row: id_map.get(row['sample_id'], row['analysis_id']),
    axis=1
)

print("✅ Sample ID corrections applied.")


✅ Sample ID corrections applied.


In [18]:
#drop unwanted/un-needed duplicates or erroneous reccords

# Drop second row where sample_id == '3497'
indices = analyses2.index[analyses2['sample_id'] == '3497'].tolist()
if len(indices) > 1:
    analyses2 = analyses2.drop(indices[1])

# Drop second row where sample_id == '3497'
indices = analyses2.index[analyses2['sample_id'] == '13817'].tolist()
if len(indices) > 1:
    analyses2 = analyses2.drop(indices[1])

# drop second row where sample_id == '8639'
indices = analyses2.index[analyses2['sample_id'] == '8639'].tolist()
if len(indices) > 1:
    analyses2 = analyses2.drop(indices[1])

#Drop second row where sample_id =='2923-2930'
# Ensure sample_id is string for consistent comparison
analyses2['sample_id'] = analyses2['sample_id'].astype(str)

# Define the sample_ids to clean duplicates for
target_ids = [str(i) for i in range(2923, 2931)]

# Identify and drop the second occurrence of each
for sid in target_ids:
    matches = analyses2.index[analyses2['sample_id'] == sid].tolist()
    if len(matches) > 1:
        # Drop the second occurrence (index 1 in the list)
        analyses2 = analyses2.drop(matches[1])
        print(f"🗑 Dropped second occurrence of sample_id {sid}")


# Ensure sample_id is string for correct matching
analyses2['sample_id'] = analyses2['sample_id'].astype(str)

# List of sample_ids to drop (as strings)
sample_ids_to_drop = [
    '4247', '4248', '4249', '4250', '4251', '4252', '4253', '4254',  # invalid samples
    '5533', '6636', '6637', '6638', '6639',
    '6875', '7361',
    '3012', '3013', '3014', '3015', '3016',
    '3049', '3050', '3051', '3052', '3053', '3054', '419', '420', '421', '422'
]

# Drop rows where sample_id is in the list
analyses2 = analyses2[~analyses2['sample_id'].isin(sample_ids_to_drop)]


🗑 Dropped second occurrence of sample_id 2923
🗑 Dropped second occurrence of sample_id 2924
🗑 Dropped second occurrence of sample_id 2925
🗑 Dropped second occurrence of sample_id 2926
🗑 Dropped second occurrence of sample_id 2927
🗑 Dropped second occurrence of sample_id 2928
🗑 Dropped second occurrence of sample_id 2929
🗑 Dropped second occurrence of sample_id 2930


In [102]:
# Ensure both 'lab_sample_id' and 'sample_id' columns exist
assert 'lab_sample_id' in analyses2.columns, "Missing column: lab_sample_id"
assert 'sample_id' in analyses2.columns, "Missing column: sample_id"

# Step 1: Ensure lab_sample_id is string so comparison works
analyses2['lab_sample_id'] = analyses2['lab_sample_id'].astype(str)

# Step 2: Manual corrections mapping
manual_corrections = {
    '66': 10376,
    '4219': 3497,
    '4221': 3498,   # same sample, different depths
    '4222': 3499,
    '4747': 16255,
    '4748': 16256,
    '5010': 15862,
    '6802': 13695,
    '7255': 8230,
    '7686': 8529,
    '6875': 3419,
    '6638': 75_1,
}

# Step 3: Apply manual corrections
for lab_id, sample_id in manual_corrections.items():
    matches = analyses2['lab_sample_id'] == lab_id
    if matches.sum() == 0:
        print(f"⚠️ Warning: lab_sample_id {lab_id} not found in data.")
    else:
        analyses2.loc[matches, 'sample_id'] = sample_id

# Step 4: Summary after correction
print("✅ Manual sample_id corrections applied.")
print(f"Remaining rows: {len(analyses2)}")
print(f"Missing sample_id entries: {analyses2['sample_id'].isna().sum()}")

# Optional: Show which rows are still missing sample_id
missing_sample_ids = analyses2[analyses2['sample_id'].isna()]
if not missing_sample_ids.empty:
    print("🔍 Rows with missing sample_id after correction:")
    print(missing_sample_ids[['lab_sample_id', 'sample_id']])


✅ Manual sample_id corrections applied.
Remaining rows: 7798
Missing sample_id entries: 0


In [20]:
# Find rows with missing sample_id
missing_sample_id_rows = analyses2[analyses2['sample_id'].isna()]

# Print how many are missing
print(f"❌ Missing sample_id in {len(missing_sample_id_rows)} rows.")

# Show the unique lab_sample_id values that are missing a sample_id
print("🔍 lab_sample_id values with missing sample_id:")
print(missing_sample_id_rows['lab_sample_id'].unique())

# Optional: Save to CSV for review
#missing_sample_id_rows.to_csv("missing_sample_ids.csv", index=False)


❌ Missing sample_id in 0 rows.
🔍 lab_sample_id values with missing sample_id:
[]


In [21]:
print("NaN sample_id rows:", analyses2['sample_id'].isna().sum())


NaN sample_id rows: 0


In [22]:
# Drop rows with missing sample_id (NaN only)
analyses2 = analyses2[analyses2['sample_id'].notna()].copy()


In [23]:
print(f"✅ Updated analyses2 has {len(analyses2)} rows.")


✅ Updated analyses2 has 7798 rows.


In [24]:
analyses3 = analyses2

In [44]:
analyses3.head()

,lab_sample_id,ID,analysis_id,horizon_id,sample_id,profile,minerology_id,soil_biology_id,EG,thick_clay,...,K,exchangable_bases_sum,CEC,V,conductivity,soluble_sodium,Min_lt_0002,Min_005_002,Min_02_005,Min_2_02
0,1,21,B_101/62_1_1,B_101/62_1_1,10999,101_62,<NA>,<NA>,NaN,61.700001,...,0.03,NaN,1.83,23.0,NaN,0.0,NaN,NaN,NaN,NaN
1,2,22,B_101/62_2_1,B_101/62_2_1,11000,101_62,<NA>,<NA>,NaN,52.799999,...,0.03,NaN,1.98,10.6,NaN,0.0,NaN,NaN,NaN,NaN
2,3,23,B_101/62_3_1,B_101/62_3_1,11001,101_62,<NA>,<NA>,NaN,42.500000,...,0.01,NaN,1.62,5.6,NaN,0.0,NaN,NaN,NaN,NaN
3,4,24,B_101/62_4_1,B_101/62_4_1,11002,101_62,<NA>,<NA>,NaN,42.599998,...,0.01,NaN,0.91,8.8,NaN,0.0,NaN,NaN,NaN,NaN
4,5,25,B_101/62_5_21,B_101/62_5_2,11003,101_62,<NA>,<NA>,NaN,36.799999,...,0.01,NaN,1.04,7.7,NaN,0.0,NaN,NaN,NaN,NaN


In [45]:
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = analyses3['sample_id'][analyses3['sample_id'].duplicated()].unique()
print(duplicated_values)

[]


In [46]:
#double checking null sample_id's
# Step 1: Ensure sample_id is a string for consistent comparison
analyses3['sample_id'] = analyses3['sample_id'].astype(str).str.strip()

# Step 2: Define mask for all "null-like" values
null_like_mask = analyses3['sample_id'].isin(['', '0', 'NULL', 'NaN', 'nan', 'None']) | analyses3['sample_id'].isna()

# Step 3: Count and list unique problematic values
null_sample_ids = analyses3[null_like_mask]['sample_id'].unique()
null_count = null_like_mask.sum()

# Step 4: Output results
print(f"🔍 Number of null-like sample_id values: {null_count}")
print("⚠️ Unique null-like sample_id values found:")
print(null_sample_ids)


🔍 Number of null-like sample_id values: 0
⚠️ Unique null-like sample_id values found:
[]


## joining elemental analyses to analyses


In [47]:
# Rename and clean sample_id as before
elemental_analyses.rename(columns={'Lab. Code': 'sample_id'}, inplace=True)

# Convert to string to clean text
sample_id_cleaned = elemental_analyses['sample_id'].astype(str)
sample_id_cleaned = sample_id_cleaned.str.replace('C-', '', regex=False)
sample_id_cleaned = sample_id_cleaned.str.replace(' MNL', '', regex=False)

# Convert to numeric with coercion (NaNs will appear here)
elemental_analyses['sample_id'] = pd.to_numeric(sample_id_cleaned, errors='coerce')

# 🔍 Show rows where sample_id is NaN
nan_sample_ids = elemental_analyses[elemental_analyses['sample_id'].isna()]
print(f"❌ Found {len(nan_sample_ids)} rows with NaN sample_id:")
print(nan_sample_ids)


❌ Found 0 rows with NaN sample_id:
Empty DataFrame
Columns: [sample_id, Soil Profile, field_sample_code, Code.1, Depht, Sampling Date, Mg, Al, Si, P, S, Cl, K, Ca, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn, As, Se, Rb, Sr, Zr, Nb, Mo, Cd, Sn, Sb, Ba, Ta, W, Pt, Au, Hg, Tl, Pb, Bi, Th, U]
Index: []

[0 rows x 44 columns]


In [48]:
# Rename 'Lab. Code' to 'sample_id'
elemental_analyses.rename(columns={'Lab. Code': 'sample_id'}, inplace=True)

# Temporarily convert to string to do the replacements
sample_id_cleaned = elemental_analyses['sample_id'].astype(str)
sample_id_cleaned = sample_id_cleaned.str.replace('C-', '', regex=False)
sample_id_cleaned = sample_id_cleaned.str.replace(' MNL', '', regex=False)

# Convert cleaned values to numeric safely
elemental_analyses['sample_id'] = pd.to_numeric(sample_id_cleaned, errors='coerce')


# Convert to integer and then string for uniformity
elemental_analyses['sample_id'] = elemental_analyses['sample_id'].astype(int).astype(str)

# Rename 'Code' to 'field_sample_code'
elemental_analyses.rename(columns={'Code': 'field_sample_code'}, inplace=True)


In [49]:
analyses2['sample_id'] = analyses3['sample_id'].astype(str)

In [50]:
print (analyses3['sample_id'].dtype)
print (elemental_analyses['sample_id'].dtype)


object
object


In [51]:
#Convert sample_id to string (no truncation unless necessary)
elemental_analyses['sample_id'] = elemental_analyses['sample_id'].astype(str).str.strip()


In [52]:
# Check which sample_ids in elemental_analyses are NOT in analyses_ready
missing_ids = elemental_analyses[~elemental_analyses['sample_id'].isin(analyses3['sample_id'])]

# Display them
print("Sample IDs in elemental_analyses not found in analyses2:")
print(missing_ids['sample_id'].unique())

print(len(missing_ids['sample_id'].unique()))



Sample IDs in elemental_analyses not found in analyses2:
['16105' '16217' '16033' '16231' '16208' '16274' '16225' '15731' '16078'
 '15786' '15693' '15678' '15959' '16372' '16418' '16459' '16477' '15498'
 '15437' '12749' '13109' '14337' '14319' '15484' '15508' '15614' '15580'
 '15582' '15606' '15463' '14496' '11343' '17892' '16976' '17269' '18248'
 '16946' '17686' '17728' '17415' '17642' '18212' '17016' '17282' '18429'
 '17340' '18824' '18448' '8721' '7313' '5502' '5392' '7317' '8626' '7193'
 '7254' '7293' '8240' '15397' '14403' '14983' '16126' '15059' '15322'
 '15268' '14808' '15106']
67


LEFT JOIN ELEMENTAL ANALYSE TO ANALYSE TABLE

In [53]:
merged1 = analyses3.merge(elemental_analyses, on='sample_id', how='left')
merged1.head()


,lab_sample_id,ID,analysis_id,horizon_id,sample_id,profile,minerology_id,soil_biology_id,EG,thick_clay,...,Ta,W,Pt,Au,Hg,Tl,Pb,Bi,Th,U
0,1,21,B_101/62_1_1,B_101/62_1_1,10999,101_62,<NA>,<NA>,NaN,61.700001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,22,B_101/62_2_1,B_101/62_2_1,11000,101_62,<NA>,<NA>,NaN,52.799999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,23,B_101/62_3_1,B_101/62_3_1,11001,101_62,<NA>,<NA>,NaN,42.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,24,B_101/62_4_1,B_101/62_4_1,11002,101_62,<NA>,<NA>,NaN,42.599998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,25,B_101/62_5_21,B_101/62_5_2,11003,101_62,<NA>,<NA>,NaN,36.799999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
print(analyses3[analyses3['sample_id'] == '6940'])
print(elemental_analyses[elemental_analyses['sample_id'] == '6940'])


     lab_sample_id    ID    analysis_id     horizon_id sample_id profile  \
1361          1362  8471  Cb_134/59_6_1  Cb_134/59_6_1      6940  134_59   

     minerology_id soil_biology_id  EG  thick_clay  ...   K  \
1361          <NA>            <NA> NaN        31.1  ... NaN   

      exchangable_bases_sum  CEC   V  conductivity  soluble_sodium  \
1361                    NaN  NaN NaN           NaN             0.0   

      Min_lt_0002  Min_005_002  Min_02_005  Min_2_02  
1361          NaN          NaN         NaN       NaN  

[1 rows x 38 columns]
   sample_id Soil Profile field_sample_code Code.1      Depht Sampling Date  \
80      6940      P134/59          A-320/59  S-492  1.30-1.70    1954-08-06   

             Mg             Al             Si      P  ...    Ta   W  Pt  Au  \
80  1457.666667  106266.333333  308859.666667  468.0  ...  26.0 NaN NaN NaN   

           Hg   Tl         Pb  Bi         Th   U  
80  10.666667  4.0  25.333333 NaN  12.333333 NaN  

[1 rows x 44 columns]


In [55]:
# Define final column list
final_columns = [
    'lab_sample_id', 'analysis_id', 'horizon_id','sample_id', 'profile', 'EG', 'thick_clay', 'fine_clay', 'silt', 'clay', 'Eq_Hum', 'atm_1/3', 'atm_15',
    'CACO3', 'gypsum', 'free_iron', 'organic_carbon', 'total_N', 'P205', 'organic_material', 'pH_H2O', 'pH_KCL',
    'Ca++', 'Mg++', 'Na+', 'K+', 'exchangable_bases_sum', 'CEC', 'V', 'conductivity', 'soluble_sodium', 'Min_<0,002',
    'Min_0,05-0,02', 'Min_0,2-0,05', 'Min_2-0,2', 
    'field_sample_code', 'Depth', 'Al', 'Si', 'P', 'S', 'Cl', 'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'As', 'Se', 'Rb', 'Sr', 'Zr', 'Nb', 'Mo', 'Cd', 'Sn', 'Sb', 'Ba', 'Ta', 'W', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi',
    'Th', 'U'
]

# Ensure all columns exist in the DataFrame
for col in final_columns:
    if col not in merged1.columns:
        merged1[col] = pd.NA  # or np.nan if preferred

# Reorder DataFrame columns
merged1 = merged1[final_columns]

# Check and print the datatypes of each column
print("Data types of each column in merged1:\n")
print(merged1.dtypes)

Data types of each column in merged1:

lab_sample_id     object
analysis_id       object
horizon_id        object
sample_id         object
profile           object
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 71, dtype: object


### Check/Change datatypes

In [56]:
# Change PK (sample_id) to a consistent datatype: Converted to string and truncate to 20 characters (similar to VARCHAR(20))
merged1['sample_id'] = merged1['sample_id'].astype(str).str.strip().str[:20]
merged1['lab_sample_id'] = merged1['lab_sample_id'].astype(str).str.strip().str[:20]

print(merged1.dtypes)

lab_sample_id     object
analysis_id       object
horizon_id        object
sample_id         object
profile           object
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 71, dtype: object


### Check and/or add FK

In [57]:
# Check if 'lab_sample_id' exists and drop it
if 'lab_sample_id' in merged1.columns:
    merged1 = merged1.drop(columns=['lab_sample_id'])

# Insert a new column, e.g., 'lab_sample_id' as a primary key starting from 1
merged1.insert(0, 'lab_sample_id', range(1, len(merged1) + 1))

### Save to CSV

In [58]:
merged1.head()

,lab_sample_id,analysis_id,horizon_id,sample_id,profile,EG,thick_clay,fine_clay,silt,clay,...,Ta,W,Pt,Au,Hg,Tl,Pb,Bi,Th,U
0,1,B_101/62_1_1,B_101/62_1_1,10999,101_62,NaN,61.700001,32.8,0.2,5.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,B_101/62_2_1,B_101/62_2_1,11000,101_62,NaN,52.799999,35.1,0.7,11.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,B_101/62_3_1,B_101/62_3_1,11001,101_62,NaN,42.500000,46.2,0.2,11.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,B_101/62_4_1,B_101/62_4_1,11002,101_62,NaN,42.599998,41.8,0.2,15.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,B_101/62_5_21,B_101/62_5_2,11003,101_62,NaN,36.799999,47.5,1.2,14.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
merged1.to_csv("/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/analyses_clean.csv", index=False)

## Filter all samples and profiles down to the matching samples and profiles in merged1 (analysis_clean)

In [99]:
# match samples and merged1 by sample_id
# filter down samples to a new df 'samples_filtered' so that samples_filtered only has the data that exists in both sample_id of merged1 and samples

# Step 1: Find the common sample_ids
common_ids = samples_check['sample_id'].isin(merged1['sample_id'])

# Step 2: Filter 'samples' to only include rows with sample_ids in 'merged1'
samples_filtered = samples_check[common_ids].copy()

# Optional: Reset index if desired
# samples_filtered.reset_index(drop=True, inplace=True)
# Now 'samples_filtered' contains only rows with matching sample_id values in both DataFrames


In [96]:
print(f"✅ merged1 has {len(merged1)} rows.")


✅ merged1 has 7798 rows.


In [98]:
print(f"✅ samples_filtered has {len(samples_filtered)} rows.")


✅ samples_filtered has 6686 rows.


In [85]:
# How many unique sample_ids are in each DataFrame?
print(f"Unique sample_ids in samples: {samples_check['sample_id'].nunique()}")
print(f"Unique sample_ids in merged1: {merged1['sample_id'].nunique()}")

# How many sample_ids in merged1 are missing from samples?
missing_ids = merged1[~merged1['sample_id'].isin(samples_check['sample_id'])]
print(f"Sample_ids in merged1 not found in samples: {missing_ids['sample_id'].nunique()}")


Unique sample_ids in samples: 14715
Unique sample_ids in merged1: 7798
Sample_ids in merged1 not found in samples: 1112


In [101]:
# Step 1: Find the common sample_ids
common_ids = samples_check['sample_id'].isin(merged1['sample_id'])

# Step 2: Filter 'samples_check' to only include rows with sample_ids in 'merged1'
samples_filtered = samples_check[common_ids].copy()

# Step 3: Diagnostics
print(f"Unique sample_ids in samples_check: {samples_check['sample_id'].nunique()}")
print(f"Unique sample_ids in merged1: {merged1['sample_id'].nunique()}")

# Step 4: Find sample_ids in merged1 that are not in samples_check
missing_ids = merged1[~merged1['sample_id'].isin(samples_check['sample_id'])]
print(f"Sample_ids in merged1 not found in samples_check: {missing_ids['sample_id'].nunique()}")

# Step 5: Export missing sample_ids to CSV
missing_ids.to_csv("missing_sample_ids.csv", index=False)
print("❗ Missing sample_ids saved to missing_sample_ids.csv")


Unique sample_ids in samples_check: 14715
Unique sample_ids in merged1: 7798
Sample_ids in merged1 not found in samples_check: 1112
❗ Missing sample_ids saved to missing_sample_ids.csv


In [93]:
site_info_clean.head()

,site_info_id,ID,profile,X_coord,Y_coord,land_cover_id,climate_id,geology_id,topo_feature_id,sampling_date,districts_id,district
0,1,2770,1_57,12.161278,-15.222598,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Namibe
1,2,48,1_59,12.575775,-4.866986,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Cabinda
2,3,1618,1_61,15.098840,-11.225411,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Cuanza Sul
3,4,881,1_63,17.081955,-9.274587,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Malanje
4,5,1750,1_64,20.788116,-11.568683,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Moxico


In [106]:
# How many unique sample_ids are in each DataFrame?
print(f"Unique profile in site_info_clean: {site_info_clean['profile'].nunique()}")
print(f"Unique profile in merged1: {merged1['profile'].nunique()}")

# How many sample_ids in merged1 are missing from samples?
missing_profs = merged1[~merged1['profile'].isin(site_info_clean['profile'])]
print(f"profile in merged1 not found in site_info_clean: {missing_profs['profile'].nunique()}")

#Export missing_profs  to CSV
missing_profs.to_csv("missing_profss.csv", index=False)
print("❗ Missing sample_ids saved to missing_profss.csv")


Unique profile in site_info_clean: 4321
Unique profile in merged1: 1587
profile in merged1 not found in site_info_clean: 8
❗ Missing sample_ids saved to missing_profs.csv


In [ ]:
## correct  profile data points in merged1 to profiles in site_info_clean based on assumptions
# make all merged1 profile become this 
#profile to change 185a_58: update to this 185_58
#Mj 26 update to 310c/63
#31_6 update to 31_63



# Soil Profile Table

In [86]:
##rename columns for clarity add FK/PK
#create copy of samples
soil_profile_record = samples_filtered.copy()
# Rename columns in the DataFrame
samples_filtered.rename(columns={
    'Registo': 'sample_id',
    'Nº Campo': 'site_info_id',
    'Ano': 'year',
    'Perfil': 'profile',
    'Campanha': 'campaign',
    'Colónia_Pais': 'country',
    'Distrito': 'district',
    'AmostraCrivada': 'sample_sifted',
    'AmostraNaoCrivada': 'sample_not_sifted',
    'Prateleira': 'shelf',
    'Sala': 'room'
}, inplace=True)

soil_profile_cleaning2 = soil_profile_record
# Add missing columns
soil_profile_cleaning2['site_info_id'] = pd.NA
soil_profile_cleaning2['sample_id'] = pd.NA 
soil_profile_cleaning2['soil_type_id'] = pd.NA 
soil_profile_cleaning2['site_info_id'] = soil_profile_cleaning2['site_info_id'].astype(str)

# Add a new Primary Key ID column starting from 1
# Check if 'lab_sample_id' exists and drop it
if 'profile_record_id' in soil_profile_cleaning2.columns:
    soil_profile_cleaning2 = soil_profile_cleaning2.drop(columns=['profile_record_id'])

# Insert a new column, e.g., 'lab_sample_id' as a primary key starting from 1
soil_profile_cleaning2.insert(0, 'profile_record_id', range(1, len(soil_profile_cleaning2) + 1))

# Reorder columns to match SAMPLES schema
profile_record_clean = soil_profile_cleaning2[[
    'profile_record_id',
    'profile',
    'site_info_id',
    'sample_id',
    'soil_type_id'
]].copy()

In [87]:
# ensure consistent data types and formatting

# correct datatypes
#profile to string
profile_record_clean['profile'] = profile_record_clean['profile'].astype(str).str.strip().str[:20]

#sample_id to string
# Convert cleaned values to numeric safely
# Convert sample_id to numeric first (coerce errors to NaN)
profile_record_clean['sample_id'] = pd.to_numeric(profile_record_clean['sample_id'], errors='coerce')
# Then convert to string (will keep NaNs as <NA>)
profile_record_clean['sample_id'] = profile_record_clean['sample_id'].astype('Int64').astype(str)


In [88]:
# check for duplicates and nulls
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = profile_record_clean['sample_id'][profile_record_clean['sample_id'].duplicated()].unique()
print(duplicated_values)

['<NA>']


In [89]:
profile_record_clean.dtypes

profile_record_id     int64
profile              object
site_info_id         object
sample_id            object
soil_type_id         object
dtype: object

In [90]:
# Preview the result
profile_record_clean.head()
#profile_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/profile_record_clean.csv", index=False)

,profile_record_id,profile,site_info_id,sample_id,soil_type_id
0,1,139,<NA>,<NA>,<NA>
1,2,139,<NA>,<NA>,<NA>
2,3,139,<NA>,<NA>,<NA>
4,4,208,<NA>,<NA>,<NA>
5,5,208,<NA>,<NA>,<NA>


# Site info table

In [91]:
import unicodedata

site_info_cleaning = profile_loc

site_info_cleaning.rename(columns={
    'PERFIL': 'profile',
    'X_COORD': 'X_coord',
    'Y_COORD': 'Y_coord', 
    'PRO': 'district'
}, inplace=True)

# Add a new Primary Key ID column starting from 1
site_info_cleaning.insert(0, 'site_info_id', range(1, len(site_info_cleaning) + 1))


# Add missing columns
site_info_cleaning['land_cover_id'] = pd.NA
site_info_cleaning['climate_id'] = pd.NA  
site_info_cleaning['geology_id'] = pd.NA
site_info_cleaning['topo_feature_id'] = pd.NA  
site_info_cleaning['sampling_date'] = pd.NA # might leave out...
site_info_cleaning['districts_id'] = pd.NA  

# Function to remove accents
def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

# Apply to all object (string) columns
for col in site_info_cleaning.select_dtypes(include='object').columns:
    site_info_cleaning[col] = site_info_cleaning[col].apply(remove_accents)

# Replace / with _ and strip/shorten 'profile' BEFORE slicing into site_info_clean
site_info_cleaning['profile'] = (
    site_info_cleaning['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str[:20]
)

# Then select columns
site_info_clean = site_info_cleaning[[
    'site_info_id',
    'ID',
    'profile',
    'X_coord',
    'Y_coord',
    'land_cover_id',
    'climate_id',
    'geology_id',
    'topo_feature_id',
    'sampling_date',
    'districts_id',
    'district'
]]


site_info_clean.head()

,site_info_id,ID,profile,X_coord,Y_coord,land_cover_id,climate_id,geology_id,topo_feature_id,sampling_date,districts_id,district
0,1,2770,1_57,12.161278,-15.222598,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Namibe
1,2,48,1_59,12.575775,-4.866986,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Cabinda
2,3,1618,1_61,15.098840,-11.225411,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Cuanza Sul
3,4,881,1_63,17.081955,-9.274587,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Malanje
4,5,1750,1_64,20.788116,-11.568683,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Moxico


# Morphology Horizon Table

In [46]:
# rename columns
morphology.rename(columns={
    'Morfo_id':'horizon_id',
    'Amostra': 'sample_id',
    'Perfil': 'profile',
    'CM':'horizon_layer',
    'Limite Superior': 'upper_depth',
    'Limite inferior': 'lower_depth',
    'Grau de humidade': 'moisture_degree',
    'Quantidade de raízes': 'root_quantity',
    'Diâmetro de raízes': 'root_diameter',
    'Textura': 'texture',
    'Tipo de estrutura': 'structure_type',
    'Classes de estrutura': 'structure_class',
    'Grau de estrutura': 'structure_degree',
    'Diâmetro de poros': 'pore_diameter',
    'Quantidade de poros': 'pore_quantity',
    'Forma de poros': 'pore_shape',
    'Cor (s)': 'dry_color_name',
    'Matiz (s)': 'dry_hue',
    'Valor (s)':'dry_value',
    'Croma (s)': 'dry_chroma',
    'Cor (h)': 'moist_color_name',
    'Matiz (h)': 'moist_hue',
    'Valor (h)': 'moist_value',
    'Croma (h)': 'moist_chroma',
    'Compacidade':'compaction',
    'Dureza': 'durability'
}, inplace=True)

# Drop unnecessary columns
morphology_cleaning = morphology.drop(columns=[
    'ID1', 'Agrupamento', 'REF', 'Pro', 'Observaçoes', 'Horizonte de diagnóstico', 'Propriedade de diagnóstico', 'Nitidez do limite', 'Designação do horizonte', 'Observaçoes', 'Confirmar', 'Adesividade', 'Plasticidade', 'Efervescência com HCl', 'Friabilidade', 'Orientação das Fendas', 'Largura das fendas', 'Quantidade de fendas'
])

#add profile_record_id to populate later from soil profile table
morphology_cleaning['profile_record_id'] = pd.NA

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
morphology_cleaning = morphology_cleaning.applymap(remove_accents)

In [47]:
# Reorder columns to match SAMPLES schema
morphology_cleaning[[
    'horizon_id',
    'sample_id',
    'profile_record_id',
    'profile',
    'horizon_layer',
    'upper_depth',
    'lower_depth',
    'moisture_degree',
    'root_quantity',
    'root_diameter',
    'texture',
    'structure_type',
    'structure_class',
    'structure_degree',
    'pore_diameter',
    'pore_quantity',
    'pore_shape',
    'dry_color_name',
    'dry_hue',
    'dry_value',
    'dry_chroma',
    'moist_color_name',
    'moist_hue',
    'moist_value',
    'moist_chroma',
    'compaction',
    'durability'
]]

# Show first few rows
morphology_cleaning.head(5)

,horizon_id,sample_id,profile,horizon_layer,upper_depth,lower_depth,dry_color_name,dry_hue,dry_value,dry_chroma,...,compaction,durability,pore_quantity,pore_diameter,pore_shape,root_quantity,root_diameter,moisture_degree,Unnamed: 46,profile_record_id
0,B_101/62_1_1,10999.0,101/62,1.0,0.0,11.0,Pardo-acinzentado a pardo,10YR,5.0,2.5,...,Pequena a minima,Brando,Pouco poroso,Muito finos,NaN,Muitas finas e bastantes medias,NaN,Seco,NaN,<NA>
1,B_101/62_2_1,11000.0,101/62,2.0,11.0,28.0,Pardo,10YR,5.0,3.0,...,Pequena,Brando,Pouco poroso,Muito finos,NaN,Bastantes finas e medias e raras grossas,NaN,Seco,NaN,<NA>
2,B_101/62_3_1,11001.0,101/62,3.0,28.0,54.0,Pardo-amarelado-claro,10YR,6.0,4.0,...,Pequena a minima,Brando,Pouco poroso,Muito finos,NaN,Algumas finas e medias e raras grossas,NaN,Seco,NaN,<NA>
3,B_101/62_4_1,11002.0,101/62,4.0,54.0,90.0,Amarelo a amarelo-avermelhado,"8,75YR",7.0,6.0,...,Pequena a minima,Brando,Pouco a medianamente poroso,Muito finos,NaN,"Poucas finas, algumas medias e raras grossas",NaN,Seco,NaN,<NA>
4,B_101/62_5_2,11003.2,101/62,5.0,90.0,160.0,Pardo-avermelhado,"7,5YR",7.0,6.0,...,Pequena,Brando,Pouco a medianamente poroso,Muito finos,NaN,Raras,Medias e grossas,Seco a humido,NaN,<NA>


In [48]:
morphology_cleaning.dtypes

horizon_id                             object
sample_id                             float64
profile                                object
horizon_layer                         float64
upper_depth                           float64
lower_depth                           float64
dry_color_name                         object
dry_hue                                object
dry_value                             float64
dry_chroma                            float64
moist_color_name                       object
moist_hue                              object
moist_value                           float64
moist_chroma                          float64
Manchas                                object
texture                                object
Abundância de elementos grosseiros     object
Forma de elementos grosseiros          object
Natureza de elementos grosseiros       object
structure_type                         object
structure_class                        object
structure_degree                  

In [49]:
# Ensure consistent data types and formatting
morphology_cleaning['profile'] = morphology_cleaning['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

#convert sample_to to string of the integer if it's a float like 11003.0
#The original string if it's not an integer or has decimal places
#"NULL" if the value is missing
def clean_sample_id(x):
    if pd.isnull(x):
        return "NULL"
    try:
        float_val = float(x)
        if float_val.is_integer():
            return str(int(float_val))
        else:
            return str(float_val)
    except:
        return str(x)

morphology_cleaning['sample_id'] = morphology_cleaning['sample_id'].apply(clean_sample_id)


In [50]:
# Check for nulls in 'sample_id' column
null_string_values = morphology_cleaning[morphology_cleaning['sample_id'] == "NULL"]['sample_id'].unique()
print(null_string_values)

null_string_count = (morphology_cleaning['sample_id'] == "NULL").sum()
print(f"🔍 Number of sample_id values equal to 'NULL': {null_string_count}")


['NULL']
🔍 Number of sample_id values equal to 'NULL': 6639


In [51]:
#sample_id(s) in morphology_cleaning but not in samples_check:
# Ensure both sample_id columns are strings for accurate comparison
morphology_cleaning['sample_id'] = morphology_cleaning['sample_id'].astype(str)
samples_check['sample_id'] = samples_check['sample_id'].astype(str)

# Get unique sample_id values in both DataFrames
morph_sample_ids = set(morphology_cleaning['sample_id'].dropna())
samples_check_ids = set(samples_check['sample_id'].dropna())

# Find sample_ids in morphology_cleaning but not in samples_check
sample_ids_not_in_check = sorted(morph_sample_ids - samples_check_ids)

# Print results
print(f"✅ Found {len(sample_ids_not_in_check)} sample_id(s) in morphology_cleaning but not in samples_check:")
print(sample_ids_not_in_check)


✅ Found 1260 sample_id(s) in morphology_cleaning but not in samples_check:
['10064.2', '1007', '1022', '1024', '10338', '10339', '10340', '10341', '10342', '10343', '10344', '10345', '10346', '10347', '10348', '10377', '104', '10428.2', '10443', '10444', '10445', '10446.2', '10448', '10449', '10450', '10451', '10467', '10468', '10469', '10470.2', '10472', '10480.2', '10485.2', '105', '10509.2', '10536.2', '10558', '10562.2', '10576.2', '106', '1067', '1068', '1069', '107', '10843.2', '10849.2', '10885.2', '1089', '1090', '10901', '10902', '10903', '10904.2', '1091', '1092', '10922.2', '1093', '10933.2', '1094', '1099', '1100.2', '11003.2', '1108.2', '1110', '1111.2', '1117', '11176.2', '1118', '1119.2', '11285.2', '11327.2', '11356.2', '11373', '11374', '1138', '1139', '11393.2', '1140.2', '1142', '1143', '1144.3', '11454.2', '1147', '1148', '11485.2', '1149.3', '11492', '11493', '11494', '1152', '1153', '1154.2', '1156', '11560.2', '1157', '1158.3', '1160', '11794.2', '1182', '1183', 

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_2227/2245887227.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples_check['sample_id'] = samples_check['sample_id'].astype(str)


In [52]:
#check which sample_id values from samples_check are missing from morphology_cleaning

missing_ids_horizon = samples_check[~samples_check['sample_id'].isin(morphology_cleaning['sample_id'])]
print("Missing sample IDs in horizon:")
print(missing_ids_horizon['sample_id'].unique())
print(len(missing_ids_horizon['sample_id'].unique()))


Missing sample IDs in horizon:
['633' '700' '707' ... '18869' '18870' '18871']
9635


**Goal** = drop any horizon_id from morphology_cleaning that does not have both:

1. Key horizon characteristics

2. A valid profile (w location and site characteristics) that appears in site_info_clean

In [53]:
import pandas as pd

# -----------------------------------------
# 1. Standardize 'profile' column formatting
# -----------------------------------------
site_info_clean.loc[:, 'profile'] = (
    site_info_clean['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str.upper()
    .str[:20]
)

morphology_cleaning.loc[:, 'profile'] = (
    morphology_cleaning['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str.upper()
    .str[:20]
)

# ---------------------------
# 2. Define valid profiles
# ---------------------------
valid_profiles = set(site_info_clean['profile'])

# ---------------------------
# 3. Define helper to check for missing/zero
# ---------------------------
def is_missing(val):
    return pd.isna(val) or val == 0 or val == '0'

# ---------------------------
# 4. Build masks for filtering
# ---------------------------

# A. Missing key horizon data
missing_key_data_mask = (
    morphology_cleaning['horizon_id'].notna() &
    morphology_cleaning['moist_color_name'].apply(is_missing) &
    morphology_cleaning['texture'].apply(is_missing) &
    morphology_cleaning['structure_type'].apply(is_missing)
)

# B. Invalid profile
invalid_profile_mask = ~morphology_cleaning['profile'].isin(valid_profiles)

# C. Missing both upper and lower depth
missing_depth_mask = (
    morphology_cleaning['upper_depth'].apply(is_missing) &
    morphology_cleaning['lower_depth'].apply(is_missing)
)

# ---------------------------
# 5. Combine all masks
# ---------------------------
rows_to_flag_mask = missing_key_data_mask | invalid_profile_mask | missing_depth_mask
rows_to_flag = morphology_cleaning[rows_to_flag_mask]

# ---------------------------
# 6. Report and drop flagged rows
# ---------------------------
print(f"⚠️ Found {len(rows_to_flag)} rows with at least one of the following issues:")
print("   • missing key horizon data")
print("   • invalid profile (not in site_info_clean)")
print("   • missing both upper_depth and lower_depth")

# Optional: Save flagged rows for review
# rows_to_flag.to_csv("/Users/inesschwartz/Desktop/morph_flagged_rows_combined.csv", index=False)

# Drop flagged rows from main DataFrame
morphology_cleaning = morphology_cleaning[~rows_to_flag_mask].copy()

# Summary after cleanup
print(f"✅ Remaining rows in morphology_cleaning: {len(morphology_cleaning)}")


⚠️ Found 2597 rows with at least one of the following issues:
   • missing key horizon data
   • invalid profile (not in site_info_clean)
   • missing both upper_depth and lower_depth
✅ Remaining rows in morphology_cleaning: 10438


In [54]:
# check for duplicates
# Make sure sample_id and horizon_id are strings
morphology_cleaning['sample_id'] = morphology_cleaning['sample_id'].astype(str).str.strip()
morphology_cleaning['horizon_id'] = morphology_cleaning['horizon_id'].astype(str).str.strip()
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = morphology_cleaning['sample_id'][morphology_cleaning['sample_id'].duplicated()].unique()
print(duplicated_values)

['11016' '11017' 'NULL' '11352' '11353' '17532' '17533' '17534' '2438'
 '7192' '13178' '20883' '9027' '9231' '8232' '9255' '8639' '8536' '4918']


In [55]:
#addressing duplicates

# -----------------------------------
# 1. Correct sample_id using horizon_id map
# -----------------------------------
# Step 1: Make sure IDs are strings and trimmed
morphology_cleaning['horizon_id'] = morphology_cleaning['horizon_id'].astype(str).str.strip()
morphology_cleaning['sample_id'] = morphology_cleaning['sample_id'].astype(str).str.strip()

# Step 2: Your mapping (horizon_id → new sample_id)
horizon_to_sample_map = {
    'UZ_24c/60_1_1': '8639',
    'UZ_24c/60_2_1': '8640',
    'UZ_24c/60_3_1': '8641',
    'UZ_24c/60_4_1': '8642',
    'UZ_24c/60_5_1': '8643',
    'UZ_66c/60_1_1': '8777',
    'UZ_66c/60_2_1': '8778',
    'UZ_66c/60_3_1': '8779',
    'UZ_66c/60_4_1': '8780',
    'CC_473/66_1_1': '17596',
    'CC_473/66_2_1': '17597',
    'CC_473/66_3_1': '17598',
    'N_85/57_1_0': '4947',
    'N_85/57_2_0': '4948',
    'N_85/57_3_0': '4949',
    'UZ_231/60_4_1': '8255',
    'B_139/61_5_1': '10353',
    'H_480/55_1_1': '3919',
    'H_91/54_1_1': '1255',
    'H_90/54_3_3': '1252',
    'Hb_136/56_1_1':'2961',
    'Hb_136/56_2_1': '2962',
    'Hb_136/56_3_1': '2963',
    'Hb_136/56_4_1': '2964',
    'UZ_216/60_5_1': '8231',
    'Bg_217/46_1_1': '694',
    'Bg_217/46_2_1': '695',
    'Bg_217/46_3_1': '696',
    'UZ_215c/60_1_1':'9255',
    'UZ_215c/60_2_1': '9256',
    'UZ_215c/60_3_1': '9257',
    'UZ_215c/60_4_1': '9258',
    'UZ_215c/60_5_1': '9259',
    'UZ_215c/60_6_1': '9260',
    'N_30/57_1_1':'4848',
    'N_30/57_2_2':'4849',
    'N_30/57_3_1':'4850',
    'UZ_66c/60_1_1': '8777',
    'UZ_66c/60_2_1': '8778',
    'UZ_66c/60_3_1': '8779',
    'UZ_66c/60_4_1': '8780',
    'UZ_66c/60_5_1': '8781',
    'UZ_66c/60_6_1': '8783',
    'UZ_66c/60_7_1': '8782',
    'H_91/54_1_1': '1255',
    'H_91/54_2_1': '1256',
    'H_91/54_3_1': '1257'
}

# Step 3: Apply the mapping to replace sample_id based on horizon_id
morphology_cleaning['sample_id'] = morphology_cleaning.apply(
    lambda row: horizon_to_sample_map.get(row['horizon_id'], row['sample_id']),
    axis=1
)


# -----------------------------------
# 2. Drop specific horizon_ids (might have already been dropped)
# -----------------------------------
horizons_to_drop = [
    'B_139/61_4_1', 'H_113/55_1_1', 'H_1306/52_1_1', 'H_1611/52_1_1',
    'H_227/46_1_1', 'H_227/46_2_1', 'H_227/46_3_1', 'H_227/46_4_1',
    'Mj_178c/63_4_1', 'Mj_321c/63_4_1', 'B_110/62_5_1', 'B_110/62_6_1'
]
morphology_cleaning = morphology_cleaning[~morphology_cleaning['horizon_id'].isin(horizons_to_drop)]


# -----------------------------------
# 3. Done — Optional: Save to file
# -----------------------------------
#df_morpho_cleaned.to_csv("/Users/inesschwartz/Desktop/df_morpho_cleaned_corrected.csv", index=False)
print("✅ Morphology data cleaned and sample_id updated via horizon_id.")


✅ Morphology data cleaned and sample_id updated via horizon_id.


In [56]:
#re-count null sample_id's
morphology_cleaning
# Check for nulls in 'sample_id' column
null_string_values2 = morphology_cleaning[morphology_cleaning['sample_id'] == "NULL"]['sample_id'].unique()
print(null_string_values2)

null_string_count2 = (morphology_cleaning['sample_id'] == "NULL").sum()
print(f"🔍 Number of sample_id values equal to 'NULL': {null_string_count2}")

['NULL']
🔍 Number of sample_id values equal to 'NULL': 4713


In [57]:
num_duplicates = morphology_cleaning['horizon_id'].duplicated().sum()
print(f"🔁 Found {num_duplicates} duplicated horizon_id(s).")


🔁 Found 0 duplicated horizon_id(s).


In [58]:
df_morpho_cleaned = morphology_cleaning

### dealing w decimals in sample_id

In [59]:
# Step 1: Ensure sample_id is a string
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].astype(str).str.strip()

# Step 2: Identify sample_ids that contain a decimal point
decimal_sample_ids = df_morpho_cleaned[df_morpho_cleaned['sample_id'].str.contains(r'\.\d+$', regex=True)]

# Step 3: Output results
print(f"🔍 Found {len(decimal_sample_ids)} sample_id(s) with decimal values.\n")

# Optional: Print the unique sample_ids with decimals
print("🧾 Affected sample_ids:")
print(decimal_sample_ids['sample_id'].unique())


🔍 Found 440 sample_id(s) with decimal values.

🧾 Affected sample_ids:
['11003.2' '11285.2' '10843.2' '10428.2' '10470.2' '10480.2' '10485.2'
 '10446.2' '10849.2' '10509.2' '10562.2' '10536.2' '11327.2' '11356.2'
 '11393.2' '11176.2' '11454.2' '10885.2' '10904.2' '10922.2' '11485.2'
 '10933.2' '4972.3' '4981.2' '5810.2' '5045.2' '5822.2' '4989.2' '5006.2'
 '5010.2' '5839.2' '4996.2' '5082.2' '5026.2' '6314.2' '6317.2' '6323.2'
 '6332.2' '6336.3' '5883.2' '6342.3' '6348.2' '6350.2' '6358.2' '5902.2'
 '5897.3' '6376.3' '6381.2' '6383.3' '6388.2' '6391.2' '6402.2' '6405.2'
 '6416.2' '5914.2' '5916.2' '6439.3' '5088.2' '5053.2' '5101.2' '5107.2'
 '5123.2' '5945.2' '5174.2' '5176.2' '5963.2' '5973.2' '5202.2' '5207.2'
 '5192.2' '5214.2' '5218.2' '5225.3' '5983.2' '5990.2' '5270.2' '5995.2'
 '5997.2' '5447.2' '6014.2' '6022.2' '5417.2' '5332.2' '6032.2' '5476.2'
 '5478.2' '5434.2' '5359.2' '5442.2' '5444.2' '5327.2' '5342.2' '5315.2'
 '5348.2' '7390.2' '7397.2' '5717.2' '5745.2' '5751.2' '573

In [60]:
# save to csv
df_morpho_cleaned.to_csv("/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/morpho_cleaned.csv")

In [61]:
## take out all decimals in sample_id (keep as just whole number)
# Function to truncate decimal part and return string
def truncate_to_int_string(x):
    try:
        return str(int(float(x)))
    except:
        return str(x).strip()  # fallback to string version if it can't convert

# Apply to sample_id column
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].apply(truncate_to_int_string)

In [62]:
#sample_id(s) in morphology_cleaning but not in samples_check:
# Ensure both sample_id columns are strings for accurate comparison
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].astype(str)
samples_check['sample_id'] = samples_check['sample_id'].astype(str)

# Get unique sample_id values in both DataFrames
morph_sample_ids = set(df_morpho_cleaned['sample_id'].dropna())
samples_check_ids = set(samples_check['sample_id'].dropna())

# Find sample_ids in morphology_cleaning but not in samples_check
sample_ids_not_in_check = sorted(morph_sample_ids - samples_check_ids)

# Print results
print(f"✅ Found {len(sample_ids_not_in_check)} sample_id(s) in morphology_cleaning but not in samples_check:")
print(sample_ids_not_in_check)


✅ Found 708 sample_id(s) in morphology_cleaning but not in samples_check:
['1024', '10338', '10339', '10340', '10341', '10342', '10343', '10344', '10345', '10346', '10347', '10348', '10377', '104', '10443', '10444', '10445', '10448', '10449', '10450', '10451', '10467', '10468', '10469', '10470', '10472', '105', '10558', '10562', '106', '107', '1089', '10901', '10902', '10903', '10904', '1091', '1092', '1094', '1099', '1100', '1108', '1110', '1111', '1117', '1119', '11373', '1138', '1140', '1142', '1144', '1147', '1149', '11492', '11493', '11494', '1152', '1154', '1156', '1158', '1160', '1182', '1183', '1184', '1186', '1187', '1188', '1189', '1190', '1191', '1198', '1200', '1204', '1205', '1206', '1208', '1209', '1211', '1212', '1213', '1215', '1217', '1219', '1220', '1222', '1223', '1224', '1225', '1227', '1229', '1231', '1233', '1234', '1235', '1236', '12450', '12451', '12452', '12494', '12495', '12496', '1250', '1252', '1255', '1256', '1257', '1274', '1275', '1286', '1287', '1288', '

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_2227/2573390024.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples_check['sample_id'] = samples_check['sample_id'].astype(str)


**consider just adding these 708 sample_id's to samples df (since they all have a related location)**

as composite keys?

deciding what to do about missing sample_id's in morphology
--> assumption: these horizons were not sampled, but just reviewed visually
--> incorrect bc there are over 9000 sample_id's from sample df that do not have a morpho_id

--> will probably just use profile record_id for visualization

In [63]:
merged1.head()

,lab_sample_id,analysis_id,horizon_id,sample_id,EG,thick_clay,fine_clay,silt,clay,Eq_Hum,...,Ta,W,Pt,Au,Hg,Tl,Pb,Bi,Th,U
0,1,B_101/62_1_1,B_101/62_1_1,10999,NaN,61.700001,32.8,0.2,5.3,4.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,B_101/62_2_1,B_101/62_2_1,11000,NaN,52.799999,35.1,0.7,11.4,6.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,B_101/62_3_1,B_101/62_3_1,11001,NaN,42.500000,46.2,0.2,11.1,6.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,B_101/62_4_1,B_101/62_4_1,11002,NaN,42.599998,41.8,0.2,15.4,5.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,B_101/62_5_21,B_101/62_5_2,11003,NaN,36.799999,47.5,1.2,14.5,7.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
# Ensure sample_id columns are strings and stripped
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].astype(str).str.strip()
merged1['sample_id'] = merged1['sample_id'].astype(str).str.strip()

# Merge to add analysis_id, sample_id (from merged1) onto df_morpho_cleaned based on horizon_id
df_morpho_enriched = df_morpho_cleaned.merge(
    merged1[['horizon_id', 'analysis_id', 'sample_id']],
    on='horizon_id',
    how='left'
)

# Rows with missing analysis_id mean df_morpho_cleaned sample_id is not in merged1
missing_analysis = df_morpho_enriched[df_morpho_enriched['analysis_id'].isna()]
print(f"Number of rows missing analysis_id after merge: {len(missing_analysis)}")

# Inspect some missing rows to understand the issue
print(missing_analysis.sample(min(10, len(missing_analysis))))


Number of rows missing analysis_id after merge: 3965
           horizon_id sample_id_x  profile  horizon_layer  upper_depth  \
1399    Cb_127/59_3_0        NULL   127_59            3.0         17.0   
7752   Mj_267c/63_1_0        NULL  267C_63            1.0          0.0   
1969    Cb_226/59_2_0        NULL   226_59            2.0         12.0   
7125    Mj_215/63_5_0        NULL   215_63            5.0        103.0   
6118    Mj_122/63_4_0        NULL   122_63            4.0         36.0   
5929   Mj_107c/63_1_0        NULL  107C_63            1.0          0.0   
10356  UZ_244c/60_2_0        NULL  244C_60            2.0          7.0   
6799    Mj_189/63_1_0        NULL   189_63            1.0          0.0   
7199    Mj_221/63_1_0        NULL   221_63            1.0          0.0   
8674    Mj_352/63_7_0        NULL   352_63            7.0        129.0   

       lower_depth                                     dry_color_name dry_hue  \
1399          31.0                                 

In [65]:
#452 duplicate horizon_id in analysis df
# are there multiple horizon_id's per analysis_id?
# is it an error that can resolved w new unique id
dupes = merged1[merged1['horizon_id'].duplicated(keep=False)]
print(f"⚠️ Duplicated horizon_ids in merged1: {dupes['horizon_id'].nunique()}")
dupes_sorted = dupes.sort_values('horizon_id')
print(dupes_sorted.head(10))


⚠️ Duplicated horizon_ids in merged1: 452
     lab_sample_id    analysis_id    horizon_id sample_id   EG  thick_clay  \
4                5  B_101/62_5_21  B_101/62_5_2     11003  NaN   36.799999   
5                6  B_101/62_5_22  B_101/62_5_2     11004  NaN   45.099998   
75              76  B_148/62_2_21  B_148/62_2_2     11285  NaN   21.799999   
76              77  B_148/62_2_22  B_148/62_2_2     11286  NaN   22.000000   
85              86   B_15/62_5_21   B_15/62_5_2     10843  NaN   27.100000   
86              87   B_15/62_5_22   B_15/62_5_2     10844  NaN   27.200001   
105            106  B_154/61_3_21  B_154/61_3_2     10335  NaN   17.799999   
106            107  B_154/61_3_22  B_154/61_3_2     10336  NaN   18.700001   
143            144  B_187/61_4_21  B_187/61_4_2     10428  1.0   31.900000   
144            145  B_187/61_4_22  B_187/61_4_2     10429  1.0   33.000000   

     fine_clay  silt  clay  Eq_Hum  ...  Ta   W  Pt  Au  Hg  Tl  Pb  Bi  Th  \
4         47.5   1.2

In [66]:
missing_in_merged1 = df_morpho_cleaned[~df_morpho_cleaned['horizon_id'].isin(merged1['horizon_id'])]


summary = pd.DataFrame({
    'total_in_morpho': [df_morpho_cleaned['horizon_id'].nunique()],
    'total_in_merged1': [merged1['horizon_id'].nunique()],
    'dupes_in_merged1': [merged1['horizon_id'].duplicated().sum()],
    'missing_in_merged1': [len(missing_in_merged1)]
})

print(summary)


   total_in_morpho  total_in_merged1  dupes_in_merged1  missing_in_merged1
0            10434              7292               506                3965


# Soil Type Table

In [67]:
# Rename columns
soil_type = soil_profile.copy()

soil_type.rename(columns={
    'Perfil': 'profile',
    'Agrupamento': 'grouping',
    'Pro': 'province',
    'País': 'country',
    'Local': 'location',
    'DATA': 'date',
    'CEP_NOME': 'CEP_NAME',
}, inplace=True)

# Drop unnecessary columns
soil_type_cleaning = soil_type.drop(columns=[
    'REF', 'province', 'country', 'location', 'DESCRITOR1', 'DESCRITOR2', 'DESCRITOR3',
    'date', 'Fase', 'D_INSERÇAO', 'Publicação', 'WRB_old', 'Missão'
], errors='ignore')  # use errors='ignore' in case some columns were already missing

# Add a new Primary Key ID column starting from 1
soil_type_cleaning.insert(0, 'soil_type_id', range(1, len(soil_type_cleaning) + 1))

# Drop accents
import unicodedata

def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

# Apply to all cells in the DataFrame
soil_type_cleaning = soil_type_cleaning.applymap(remove_accents)

# Replace / with _ and strip/shorten 'profile' BEFORE slicing into site_info_clean
soil_type_cleaning['profile'] = (
    soil_type_cleaning['profile']
    .astype(str)
    .str.replace('/', '_')
    .str.strip()
    .str[:20]
)

# Keep only relevant columns
soil_type_clean = soil_type_cleaning[[
    'soil_type_id',
    'profile',
    'CEP_GR',
    'CEP_NAME',
    'FAO'
]]

# Preview
soil_type_clean.head(2)


,soil_type_id,profile,CEP_GR,CEP_NAME,FAO
0,1,1_51,NaN,NaN,NaN
1,2,1_57,Aridicos,Aridicos com calcario Pardo-cinzentos,CLha


In [68]:
# check duplicates
# Filter out and print duplicated samples (including the first occurrence)
duplicated_values_st = soil_type_clean['profile'][soil_type_clean['profile'].duplicated()].unique()
print(duplicated_values_st)


[]


In [69]:
# Check for profile values that are the string "NULL"
null_string_values_st = soil_type_clean[soil_type_clean['profile'] == "NULL"]['profile'].unique()
print("🔍 Unique 'profile' values equal to string 'NULL':", null_string_values_st)

null_string_count_st = (soil_type_clean['profile'] == "NULL").sum()
print(f"🔢 Number of 'profile' values equal to string 'NULL': {null_string_count_st}")


🔍 Unique 'profile' values equal to string 'NULL': []
🔢 Number of 'profile' values equal to string 'NULL': 0


In [70]:
import pandas as pd

# ---------------------------------------
# 1. Define cleaning function for profile
# ---------------------------------------
def clean_profile_column(series):
    return (
        series.astype(str)
        .str.replace('/', '_')
        .str.strip()
        .str.upper()
        .str[:20]
    )

# ---------------------------------------
# 2. Safely clean 'profile' columns
# ---------------------------------------
site_info_clean = site_info_clean.copy()
soil_type_clean = soil_type_clean.copy()

site_info_clean.loc[:, 'profile'] = clean_profile_column(site_info_clean['profile'])
soil_type_clean.loc[:, 'profile'] = clean_profile_column(soil_type_clean['profile'])

# ---------------------------------------
# 3. Create sets of unique profiles
# ---------------------------------------
site_profiles = set(site_info_clean['profile'].unique())
soil_profiles = set(soil_type_clean['profile'].unique())

# ---------------------------------------
# 4. Compare overlaps and mismatches
# ---------------------------------------
only_in_site_info = site_profiles - soil_profiles
only_in_soil_type = soil_profiles - site_profiles
in_both = site_profiles & soil_profiles

# ---------------------------------------
# 5. Report results
# ---------------------------------------
print(f"Total profiles in site_info_clean: {len(site_profiles)}")
print(f"Total profiles in soil_type_clean: {len(soil_profiles)}")
print()
print(f"✅ Profiles in both: {len(in_both)}")
print(f"❌ Profiles only in site_info_clean: {len(only_in_site_info)} "
      f"({len(only_in_site_info) / len(site_profiles) * 100:.1f}%)")
print(f"❌ Profiles only in soil_type_clean: {len(only_in_soil_type)} "
      f"({len(only_in_soil_type) / len(soil_profiles) * 100:.1f}%)")


Total profiles in site_info_clean: 4321
Total profiles in soil_type_clean: 2518

✅ Profiles in both: 2376
❌ Profiles only in site_info_clean: 1945 (45.0%)
❌ Profiles only in soil_type_clean: 142 (5.6%)


In [71]:
#export for investigation

unmatched_site_info = site_info_clean[site_info_clean['profile'].isin(only_in_site_info)]
unmatched_site_info.to_csv("unmatched_profiles_site_info.csv", index=False)


Save clean table

# Climate table


In [72]:
# Load original Excel data
profile_loc = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_local.xlsx")

# Create a copy for cleaning
climate_features_cleaning = profile_loc.copy()

# Mapping from climate codes to descriptions
code_to_description = {
    "B1": "Húmido",
    "B2": "Húmido",
    "B3": "Húmido",
    "B4": "Húmido",
    "C1": "Sub-húmido seco",
    "C2": "Sub-húmido chuvoso",
    "D": "Semi-árido",
    "E": "Árido",
    "Aw": "Tropical chuvoso com estação seca no Inverno, de savana",
    "BSw": "Seco de estepe, com chuva predominante no Verão",
    "BWw": "Seco de deserto, com chuva predominante no Verão",
    "Cw": "Mesotérmico húmido com estação seca no Inverno",
    "ARe": "Arídico extremo",
    "ARf": "Arídico fraco",
    "ARt": "Arídico típico",
    "tUDs": "Tempúdico seco",
    "tUSh": "Tempústico húmido",
    "tUSt": "Tempústico típico",
    "TUDs": "Tropúdico seco",
    "TUSa": "Tropústico arídico",
    "TUSu": "Tropústico údico",
    "TUSt": "Tropustico típico",
    "H": "Hipertérmico",
    "iH": "Iso-Hipertérmico",
    "iT": "Iso-Térmico",
    "T": "Térmico"
}

# Replace climate codes with descriptions
for col in ["CL_THORNTH", "CL_KOPPEN", "REG_HÍDRIC", "REG_TÉRMIC"]:
    climate_features_cleaning[col] = climate_features_cleaning[col].replace(code_to_description)

# Function to average values like "21-22" -> 21.5
def average_range(value):
    if isinstance(value, str) and '-' in value:
        try:
            nums = [float(x.strip()) for x in value.split('-')]
            return sum(nums) / len(nums)
        except:
            return None
    try:
        return float(value)
    except:
        return None

# Apply to temperature and precipitation columns
climate_features_cleaning["TMA"] = climate_features_cleaning["TMA"].apply(average_range)
climate_features_cleaning["PMA"] = climate_features_cleaning["PMA"].apply(average_range)

# Rename columns for clarity
climate_features_cleaning.rename(columns={
    'PERFIL': 'profile',
    'CL_THORNTH': 'thornthwaite_climate',
    'CL_KOPPEN': 'koppen_climate',
    'TMA': 'mean_annual_temp',
    'PMA': 'mean_annual_precip',
    'REG_HÍDRIC': 'hydric_regime',
    'REG_TÉRMIC': 'thermal_regime'
}, inplace=True)

# ensure consistent profile formatting
climate_features_cleaning['profile'] = climate_features_cleaning['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

# Drop accents from text values
def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

climate_features_cleaning = climate_features_cleaning.applymap(remove_accents)

# Add primary key column
climate_features_cleaning.insert(0, 'climate_id', range(1, len(climate_features_cleaning) + 1))

# Replace empty strings or nulls in numeric columns with \N (Postgres null)
for col in ['mean_annual_temp', 'mean_annual_precip']:
    climate_features_cleaning[col] = climate_features_cleaning[col].replace(
        ['', ' ', 'NULL', None, pd.NA, pd.NaT, 'nan', float('nan')], ''
    )

# Select and reorder relevant columns for export
climate_features_clean = climate_features_cleaning[[
    'climate_id',
    'profile',
    'ID',
    'mean_annual_temp',
    'mean_annual_precip',
    'koppen_climate',
    'thornthwaite_climate',
    'hydric_regime',
    'thermal_regime'
]]

# Preview
climate_features_clean.head()


,climate_id,profile,ID,mean_annual_temp,mean_annual_precip,koppen_climate,thornthwaite_climate,hydric_regime,thermal_regime
0,1,1_57,2770,,,NaN,Arido,NaN,NaN
1,2,1_59,48,,,NaN,Humido,NaN,NaN
2,3,1_61,1618,,,NaN,NaN,NaN,NaN
3,4,1_63,881,21.5,1500.0,"Tropical chuvoso com estacao seca no Inverno, ...",Humido,Tropustico udico,Iso-Hipertermico
4,5,1_64,1750,,,NaN,NaN,NaN,NaN


Check FK relationships and datatypes

In [73]:
# Check and print the datatypes of each column
print("Data types of each column in profile_clean:\n")
print(climate_features_clean.dtypes)

Data types of each column in profile_clean:

climate_id               int64
profile                 object
ID                       int64
mean_annual_temp        object
mean_annual_precip      object
koppen_climate          object
thornthwaite_climate    object
hydric_regime           object
thermal_regime          object
dtype: object


# Topo features table

In [74]:
# Create a copy for cleaning
topo_features_cleaning = profile_loc.copy()

# Rename columns for clarity
topo_features_cleaning.rename(columns={
    'PERFIL': 'profile',
    'TOPOGRAFIA': 'slope_code',
    'ALTITUDE': 'altitude',
}, inplace=True)

# ensure consistent profile formatting
topo_features_cleaning['profile'] = topo_features_cleaning['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

# Add missing columns
topo_features_cleaning['aspect'] = pd.NA  
topo_features_cleaning['land_surface_temp'] = pd.NA
topo_features_cleaning['dem_elevation'] = pd.NA  

# Add primary key column
topo_features_cleaning.insert(0, 'topo_features_id', range(1, len(topo_features_cleaning) + 1))

# Create slope class mapping dictionary
slope_code_to_description = {
    "D1": "Plano (Declives < 2%)",
    "D2": "Ondulado muito suave (Declives > 2% e < 3%)",
    "D3": "Ondulado suave (Declives > 3% e < 5%)",
    "D4": "Ondulado (Declives > 5% e < 8%)",
    "D5": "Acidentado (Declives > 8% e < 15%)",
    "D6": "Escarpado (Declives >15% e < 30%)",
    "D7": "Montanhoso (Declives > 30%)"
}

# Create a mapping DataFrame for slope classes
slope_classes_df = pd.DataFrame([
    {"slope_code": code, "slope_description": desc}
    for code, desc in slope_code_to_description.items()
])

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
topo_features_cleaning = topo_features_cleaning.applymap(remove_accents)

# Final cleaned topo features table (referencing slope_code, not description)
topo_features_clean = topo_features_cleaning[[
    'topo_features_id',
    'ID',
    'profile', 
    'slope_code',
    'altitude',
    'aspect',
    'land_surface_temp',
    'dem_elevation'
]]

# Preview
topo_features_clean.head()


,topo_features_id,ID,profile,slope_code,altitude,aspect,land_surface_temp,dem_elevation
0,1,2770,1_57,NaN,32.0,<NA>,<NA>,<NA>
1,2,48,1_59,D5,NaN,<NA>,<NA>,<NA>
2,3,1618,1_61,NaN,NaN,<NA>,<NA>,<NA>
3,4,881,1_63,D1,1210.0,<NA>,<NA>,<NA>
4,5,1750,1_64,NaN,NaN,<NA>,<NA>,<NA>


# Geological features table

In [75]:
# Load cleaned geo features data
geo_features_cleaning = pd.read_excel("/Users/inesschwartz/GreenDataScience/Thesis/tables_soil_database/Perfis_local.xlsx")

# Rename columns
geo_features_cleaning.rename(columns={
    'PERFIL': 'profile',
    'GEOLOGIA': 'geology_id',
    'LITOLOGIA': 'lithology_id',
    'LITOLOGIA_1954': 'lithology_1954_id',
}, inplace=True)

# Add primary key column
geo_features_cleaning.insert(0, 'geo_features_id', range(1, len(geo_features_cleaning) + 1))

# Final normalized geo_features table (with codes as foreign keys)
geo_features_clean = geo_features_cleaning[[
    'geo_features_id',
    'profile',
    'ID',
    'geology_id',
    'lithology_id',
    'lithology_1954_id'
]]

# ensure consistent profile formatting
geo_features_clean['profile'] = geo_features_clean['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
geo_features_clean = geo_features_clean.applymap(remove_accents)

#preview
geo_features_clean.head()

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_2227/2818143663.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_features_clean['profile'] = geo_features_clean['profile'].astype(str).str.replace('/', '_').str.strip().str[:20]


,geo_features_id,profile,ID,geology_id,lithology_id,lithology_1954_id
0,1,1_57,2770,NaN,d,NaN
1,2,1_59,48,Oendolongo,pp,Sistema do Maiombe
2,3,1_61,1618,NaN,NaN,NaN
3,4,1_63,881,Karroo,Cs/Cal,Serie de Cassanje - T2'T1
4,5,1_64,1750,NaN,NaN,NaN


In [76]:
## mapping tables
# Mappings for geology
geology_mapping = {
    "Kalahari": "Sistema do Kalahari",
    "Superficiais": "Formações Superficiais",
    "Karroo": "Sistema do Karroo",
    "Bembe": "Sistema do Bembe",
    "Oendolongo": "Sistema do Oendolongo",
    "Base": "Complexo de base",
    "Proterozóico": "Proterozóico",
    "Pleistocénico": "Pleistocénico",
    "Terciário": "Terciário (médio e inferior)",
    "TQ": "Quaternário e Terciário superior",
    "Cretácio": "",  # no description provided
    "RPKS": "Recente Plistocénico e Kalahari Superior"
}

# Mappings for lithology_1954
lithology_1954_mapping = {
    "γ": "Granitos, Granodioritos e Quartzodioritos",
    "PL": "Xistos, metaquartzitos, conglomerados, arcoses, ect.",
    "λ": "Rochas eruptivas indeterminadas",
    "δp": "Doleritos, doleritos pigeoníticos",
    "δab": "Diabases, diabases albito-cloriticas",
    "ε": "Noritos, gabros e peridotitos",
    "JK": "Composto de conglomerados, areias, cascalhos do Kalahari",
    "C": "Série Xisto - calcária",
    "Cal": "Sedimentos arenosos não consolidados",
    "K": "Série xisto - gresosa",
    "RT": "Não diferenciado",
    "σ": "Sienitos, sienitos nefelínicos",
    "Q": "Depósitos fossilíferos",
    "CS": "Grande conglomerado e série de Mwashya"
}

# Mappings for lithology
lithology_mapping = {
    "a": "Rochas arenáceas consolidadas",
    "aq": "Grés quartzíticos do Oendolongo",
    "b": "Rochas eruptivas básicas",
    "c": "Rochas sedimentares consolidadas calcárias",
    "c'": "Rochas sedimentares não consolidadas calcárias",
    "cg": "Rochas cristalofílicas argiláceas",
    "d": "Sedimentos não consolidados de origem marinha",
    "dc": "Depósitos coluvionares",
    "dr": "Diorítos",
    "e": "Rochas sedimentares consolidadas não calcárias",
    "g": "Rochas argiláceas consolidadas não calcárias",
    "g'": "Rochas argiláceas não consolidadas não calcárias",
    "g''": "Rochas cristalinas pouco micas em quartzo",
    "gp": "Rochas do  complexo gabro-plagioclastíco",
    "k": "Sedimentos não consolidados grosseiros do Kalahari",
    "m": "Rochas sedimentares não consolidadas calco-gipsíferas",
    "m'": "Depósitos coluvionares margosos",
    "mm": "Materiais mistos",
    "n": "Sedimentos não consolidados de origem continental",
    "nd": "não descrito",
    "pp": "Sedimentos não consolidados grosseiros plio-plistocénicos",
    "q": "Rochas cristalinas quartzíferas",
    "q'": "Materiais redistribuídos provenientes de desagregação rochas crist. quartzíferas",
    "qf": "Quartzitos ferruginosos do Oendolongo",
    "r": "Sedimentos grosseiros não especificados",
    "s": "Sienitos",
    "sx": "Formações (ou rochas) sedimentares não especificadas",
    "sx1": "Rochas sedimentares consolidadas com e sem calcário",
    "sx2": "Rochas sedimentares consolidadas",
    "v": "Materiais vulcânicos",
    "v'": "Rochas do complexo alcalino e/ou carboatítico",
    "x": "Rochas consolidadas não especificadas",
    "xm": "Xistos metamórficos",
    "xq": "Rochas cristalinas não especificadas",
    "z": "Rochas metassedimentares"
}

# Save each mapping as a DataFrame
pd.DataFrame([
    {"geology_code": k, "geology_description": v}
    for k, v in geology_mapping.items()
]).to_csv("/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/geology_mapping.csv", index=False)

pd.DataFrame([
    {"lithology_code": k, "lithology_description": v}
    for k, v in lithology_mapping.items()
]).to_csv("/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/lithology_mapping.csv", index=False)

pd.DataFrame([
    {"lithology_1954_code": k, "lithology_1954_description": v}
    for k, v in lithology_1954_mapping.items()
]).to_csv("/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/lithology1954_mapping.csv", index=False)


# District table

**don't need**

Why do I need to make a separate table??

# Minerology info table

# Biology Info table

# Checking CSV Datatypes before DB export

In [77]:
import pandas as pd
import os

# Set the path to your folder containing the 10 CSV files
csv_folder = "/Users/inesschwartz/GreenDataScience/Thesis/tables_clean"  

# List all CSV files in the folder
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

# Loop through each file and display column data types
for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    print(f"\n📄 File: {file}")
    
    try:
        df = pd.read_csv(file_path)
        print(df.dtypes)
    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")


📄 File: analyses.csv
lab_sample_id      int64
sample_id        float64
EG               float64
thick_clay       float64
fine_clay        float64
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 68, dtype: object

📄 File: soil_type_clean.csv
soil_type_id     int64
profile         object
CEP_GR          object
CEP_NAME        object
FAO             object
dtype: object

📄 File: analyses_clean.csv
lab_sample_id      int64
analysis_id       object
horizon_id        object
sample_id         object
EG               float64
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 70, dtype: object

📄 File: profile_record.csv
profile_record_id      int64
profile               object
site_info_id         float64
soil_type_id         float64
sample_id              int64
dtype: object

📄 

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_2227/3654665737.py:16: DtypeWarning: Columns (13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


horizon_id            object
sample_id            float64
profile_record_id    float64
horizon_layer        float64
upper_depth          float64
lower_depth          float64
moisture_degree       object
root_quantity         object
root_diameter         object
texture               object
structure_type        object
structure_class       object
structure_degree      object
pore_diameter         object
pore_quantity         object
pore_shape            object
dry_color_name        object
dry_hue               object
dry_value            float64
dry_chroma           float64
moist_color_name      object
moist_hue             object
moist_value          float64
moist_chroma         float64
compaction            object
durability            object
dtype: object

📄 File: site_info.csv
site_info_id          int64
profile              object
X_coord             float64
Y_coord             float64
district             object
geo_features_id       int64
climate_id            int64
topo_features

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_2227/3654665737.py:16: DtypeWarning: Columns (13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


# Foreign key imports and datatype consistency

In [78]:
## standardize datatypes for identifiers
def convert_identifiers_to_string(df, id_columns):
    """
    Converts specified identifier columns in a DataFrame to string type,
    safely handling float64 values and preserving missing values (NA).
    Avoids SettingWithCopyWarning.
    """
    df = df.copy()  # ensure we're working with a copy, not a slice
    for col in id_columns:
        if col in df.columns:
            df.loc[:, col] = df[col].apply(
                lambda x: str(int(x)) if pd.notna(x) and isinstance(x, float) and x.is_integer()
                else str(x) if pd.notna(x)
                else pd.NA
            ).astype("string")
    return df

#usage

# Define identifier columns
identifier_columns = [
    'sample_id', 'site_info_id', 'profile',
    'horizon_id', 'lab_sample_id', 'lab_sample_id', 'climate_id', 'geo_features_id', 'topo_features_id', 'profile_record_id'
]

# Apply to each relevant dataframe
samples_check = convert_identifiers_to_string(samples_check, identifier_columns)
df_morpho_cleaned = convert_identifiers_to_string(df_morpho_cleaned, identifier_columns)
profile_clean = convert_identifiers_to_string(profile_record_clean, identifier_columns)
merged1 = convert_identifiers_to_string(merged1, identifier_columns)
site_info_clean = convert_identifiers_to_string(site_info_clean, identifier_columns)
#district_clean = convert_identifiers_to_string(district_clean, identifier_columns)
soil_type_clean = convert_identifiers_to_string(soil_type_clean, identifier_columns)
geo_features_clean = convert_identifiers_to_string(geo_features_clean, identifier_columns)
topo_features_clean = convert_identifiers_to_string(topo_features_clean, identifier_columns)


In [79]:
# Check types of identifier columns in each dataframe
dfs = {
    "samples": samples_check,
    "horizon": df_morpho_cleaned,
    "profile": profile_clean,
    "lab_analysis": merged1,
    "site_info": site_info_clean,
    #"district_clean": district_clean,
    "soil_type": soil_type_clean,
    "geo_features": geo_features_clean,
    "topo_features": topo_features_clean,
}

for name, df in dfs.items():
    print(f"\n{name} column types:")
    for col in identifier_columns:
        if col in df.columns:
            print(f"  {col}: {df[col].dtype}")



samples column types:
  sample_id: object
  site_info_id: object
  profile: object
  horizon_id: object
  profile_record_id: object

horizon column types:
  sample_id: object
  profile: object
  horizon_id: object
  profile_record_id: object

profile column types:
  sample_id: object
  site_info_id: object
  profile: object
  profile_record_id: string

lab_analysis column types:
  sample_id: object
  horizon_id: object
  lab_sample_id: string
  lab_sample_id: string

site_info column types:
  site_info_id: string
  profile: object
  climate_id: object

soil_type column types:
  profile: object

geo_features column types:
  profile: object
  geo_features_id: string

topo_features column types:
  profile: object
  topo_features_id: string


In [80]:
## validate FK references in tables

samples_check['sample_id'] = samples_check['sample_id'].astype(str).str.strip().str.replace('.0', '')
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].astype(str).str.strip().str.replace('.0', '')

# Check for missing FK references in morphology
missing_samples = samples_check[~samples_check['sample_id'].isin(df_morpho_cleaned['sample_id'])]

print(f"{len(missing_samples)} sample_id(s) in samples_check are missing in horizon.")
print(missing_samples[['sample_id']])


9704 sample_id(s) in samples_check are missing in horizon.
      sample_id
1           631
3           633
7           697
8           698
9           699
...         ...
14710     18867
14711     18868
14712     18869
14713     18870
14714     18871

[9704 rows x 1 columns]


In [81]:
#inspecting a random 25 samples that are missing in horizon

# Print random sample of 25 missing sample_ids
sample_subset = missing_samples.sample(n=25, random_state=42)  # Set random_state for reproducibility
print(sample_subset[['sample_id']].reset_index(drop=True))



   sample_id
0       2044
1      15936
2      17780
3       5288
4      13422
5      10276
6      12228
7       9094
8      15374
9       7359
10      1762
11     17363
12     12169
13     18754
14      2236
15      9709
16     16539
17     17941
18     14442
19      7699
20     11951
21     15690
22     11558
23      1823
24      6124


In [82]:
# Samples table horizon_id FK 

# Drop old horizon_id if it exists (to avoid confusion)
samples_check = samples_check.drop(columns=['horizon_id'], errors='ignore')

samples_check['sample_id'] = samples_check['sample_id'].astype(str).str.strip().str.replace('.0', '', regex=False)
df_morpho_cleaned['sample_id'] = df_morpho_cleaned['sample_id'].astype(str).str.strip().str.replace('.0', '', regex=False)


# Merge horizon_id from morph into samples
samples_check = samples_check.merge(
    df_morpho_cleaned[['sample_id', 'horizon_id']],
    on='sample_id',
    how='left'
)

missing = samples_check[samples_check['horizon_id'].isna()]
print(f"{len(missing)} samples could not be matched with a horizon_id.")


9704 samples could not be matched with a horizon_id.


In [83]:
# Re-order and select relevant columns
samples_clean1 = samples_check[[
    'sample_id',
    'site_info_id', 
    'profile',
    'horizon_id',
    'year',
    'shelf',
    'room'  # Ensure this matches the column name in your DataFrame
]].copy()

# Preview the result
samples_clean1.head()


,sample_id,site_info_id,profile,horizon_id,year,shelf,room
0,630,172,139,Hb_139/46_1_1,1946.0,1,22
1,631,173,139,NaN,1946.0,1,22
2,632,174,139,Hb_139/46_3_1,1946.0,1,22
3,633,175,139,NaN,1946.0,1,22
4,687,1034,208,Hb_208/46_1_1,1946.0,1,22


In [84]:
samples_clean1.head()

,sample_id,site_info_id,profile,horizon_id,year,shelf,room
0,630,172,139,Hb_139/46_1_1,1946.0,1,22
1,631,173,139,NaN,1946.0,1,22
2,632,174,139,Hb_139/46_3_1,1946.0,1,22
3,633,175,139,NaN,1946.0,1,22
4,687,1034,208,Hb_208/46_1_1,1946.0,1,22


## completing profile_record_clean table

In [85]:
profile_record_clean.head()

,profile_record_id,profile,site_info_id,sample_id,soil_type_id
0,1,139,<NA>,<NA>,<NA>
1,2,139,<NA>,<NA>,<NA>
2,3,139,<NA>,<NA>,<NA>
3,4,139,<NA>,<NA>,<NA>
4,5,208,<NA>,<NA>,<NA>


In [86]:
# Drop old foreign key columns if they exist
profile_record_clean = profile_record_clean.drop(
    columns=['site_info_id', 'sample_id', 'soil_type_id'], errors='ignore'
)

# Merge site_info_id (one-to-one or many-to-one) using 'ID'
profile_record_clean = profile_record_clean.merge(
    site_info_clean[['profile', 'site_info_id']],
    on='ID',
    how='left'
)

# Merge soil_type_id (one-to-one or many-to-one) using 'profile'
profile_record_clean = profile_record_clean.merge(
    soil_type_clean[['profile', 'soil_type_id']],
    on='profile',
    how='left'
)

# Merge sample_id (one-to-one or many-to-one) using 'profile'
profile_record_clean = profile_record_clean.merge(
    samples_clean1[['profile', 'sample_id']],
    on='profile',
    how='left'
)

#drop PK and replace w new profile_record_id PK
# Drop old FK columns if they exist
profile_record_clean = profile_record_clean.drop(
    columns=['profile_record_id' ],
    errors='ignore'
)

# Ensure primary key 'profile_record_id' exists
if 'profile_record_id' not in profile_record_clean.columns:
    profile_record_clean.insert(0, 'profile_record_id', range(1, len(profile_record_clean) + 1))

profile_record_clean['profile_record_id'] = profile_record_clean['profile_record_id'].astype(str)

# Final column order
column_order = [
    'profile_record_id',
    'profile',
    'site_info_id',
    'soil_type_id',
    'sample_id'
]
profile_record_clean = profile_record_clean[column_order]

# Check for missing links (optional sanity check)
missing_site = profile_record_clean[profile_record_clean['site_info_id'].isna()]
missing_soil = profile_record_clean[profile_record_clean['soil_type_id'].isna()]

print(f"{len(missing_site)} profiles missing site_info_id")
print(f"{len(missing_soil)} profiles missing soil_type_id")

profile_record_clean.head()

KeyError: 'ID'

In [118]:
df_morpho_cleaned.head()

,horizon_id,sample_id,profile,horizon_layer,upper_depth,lower_depth,dry_color_name,dry_hue,dry_value,dry_chroma,...,compaction,durability,pore_quantity,pore_diameter,pore_shape,root_quantity,root_diameter,moisture_degree,Unnamed: 46,profile_record_id
0,B_101/62_1_1,10999,101_62,1.0,0.0,11.0,Pardo-acinzentado a pardo,10YR,5.0,2.5,...,Pequena a minima,Brando,Pouco poroso,Muito finos,NaN,Muitas finas e bastantes medias,NaN,Seco,NaN,<NA>
1,B_101/62_2_1,11000,101_62,2.0,11.0,28.0,Pardo,10YR,5.0,3.0,...,Pequena,Brando,Pouco poroso,Muito finos,NaN,Bastantes finas e medias e raras grossas,NaN,Seco,NaN,<NA>
2,B_101/62_3_1,11001,101_62,3.0,28.0,54.0,Pardo-amarelado-claro,10YR,6.0,4.0,...,Pequena a minima,Brando,Pouco poroso,Muito finos,NaN,Algumas finas e medias e raras grossas,NaN,Seco,NaN,<NA>
3,B_101/62_4_1,11002,101_62,4.0,54.0,90.0,Amarelo a amarelo-avermelhado,"8,75YR",7.0,6.0,...,Pequena a minima,Brando,Pouco a medianamente poroso,Muito finos,NaN,"Poucas finas, algumas medias e raras grossas",NaN,Seco,NaN,<NA>
4,B_101/62_5_2,11003,101_62,5.0,90.0,160.0,Pardo-avermelhado,"7,5YR",7.0,6.0,...,Pequena,Brando,Pouco a medianamente poroso,Muito finos,NaN,Raras,Medias e grossas,Seco a humido,NaN,<NA>


In [122]:
# Convert to string dtype in both DataFrames before merge
df_morpho_cleaned['profile_record_id'] = df_morpho_cleaned['profile_record_id'].astype(str)
profile_record_clean['profile_record_id'] = profile_record_clean['profile_record_id'].astype(str)

#drop old FK
# Drop old FK columns if they exist
df_morpho_cleaned = df_morpho_cleaned.drop(
    columns=['profile_record_id'],
    errors='ignore'
)
# Now merge on 'profile_record_id'
df_morpho_cleaned = df_morpho_cleaned.merge(
    profile_record_clean[['profile', 'profile_record_id']],
    on='profile',
    how='left'
)

df_morpho_cleaned1 = df_morpho_cleaned[[
    'horizon_id',
    'sample_id',
    #'profile_record_id',
    'profile',
    'horizon_layer',
    'upper_depth',
    'lower_depth',
    'moisture_degree',
    'root_quantity',
    'root_diameter',
    'texture',
    'structure_type',
    'structure_class',
    'structure_degree',
    'pore_diameter',
    'pore_quantity',
    'pore_shape',
    'dry_color_name',
    'dry_hue',
    'dry_value',
    'dry_chroma',
    'moist_color_name',
    'moist_hue',
    'moist_value',
    'moist_chroma',
    'compaction',
    'durability'
]]

In [123]:
df_morpho_cleaned1.head()



,horizon_id,sample_id,profile,horizon_layer,upper_depth,lower_depth,moisture_degree,root_quantity,root_diameter,texture,...,dry_color_name,dry_hue,dry_value,dry_chroma,moist_color_name,moist_hue,moist_value,moist_chroma,compaction,durability
0,B_101/62_1_1,10999,101_62,1.0,0.0,11.0,Seco,Muitas finas e bastantes medias,NaN,Arenoso,...,Pardo-acinzentado a pardo,10YR,5.0,2.5,Pardo-acinzentado-escuro,10YR,4.0,2.0,Pequena a minima,Brando
1,B_101/62_2_1,11000,101_62,2.0,11.0,28.0,Seco,Bastantes finas e medias e raras grossas,NaN,Arenoso-franco,...,Pardo,10YR,5.0,3.0,Pardo-amarelado-escuro,10YR,3.0,4.0,Pequena,Brando
2,B_101/62_3_1,11001,101_62,3.0,28.0,54.0,Seco,Algumas finas e medias e raras grossas,NaN,Arenoso-franco,...,Pardo-amarelado-claro,10YR,6.0,4.0,Pardo-amarelado-escuro,10YR,4.0,4.0,Pequena a minima,Brando
3,B_101/62_4_1,11002,101_62,4.0,54.0,90.0,Seco,"Poucas finas, algumas medias e raras grossas",NaN,Franco-arenoso a arenoso-franco,...,Amarelo a amarelo-avermelhado,"8,75YR",7.0,6.0,Pardo-forte,"7,5YR",5.0,6.0,Pequena a minima,Brando
4,B_101/62_5_2,11003,101_62,5.0,90.0,160.0,Seco a humido,Raras,Medias e grossas,Arenoso-franco,...,Pardo-avermelhado,"7,5YR",7.0,6.0,Pardo-forte,"7,5YR",5.0,6.0,Pequena,Brando


In [126]:
df_morpho_cleaned1.to_csv('/Users/inesschwartz/Desktop/morpho_clean.csv')

## Completing site_info_clean table

In [42]:
# Convert keys to string before merge to avoid dtype mismatch errors
site_info_clean['site_info_id'] = site_info_clean['ID'].astype(str)
geo_features_clean['ID'] = geo_features_clean['ID'].astype(str)
climate_features_clean['ID'] = climate_features_clean['ID'].astype(str)
topo_features_clean['ID'] = topo_features_clean['ID'].astype(str)

/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_8750/4452788.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_features_clean['ID'] = climate_features_clean['ID'].astype(str)


In [43]:
# Drop old FK columns if they exist
site_info_clean = site_info_clean.drop(
    columns=['geo_features_id', 'climate_id', 'topo_features_id', 'district_id', 'land_cover_id', 'geology_id','topo_feature_id','sampling_date', 'districts_id' ],
    errors='ignore'
)

In [44]:
# --- Step 1: Convert 'ID' columns to string ---
site_info_clean['ID'] = site_info_clean['ID'].astype(str)
geo_features_clean['ID'] = geo_features_clean['ID'].astype(str)
climate_features_clean['ID'] = climate_features_clean['ID'].astype(str)
topo_features_clean['ID'] = topo_features_clean['ID'].astype(str)

# --- Step 2: Merge feature tables into site_info_clean ---
site_info_clean = site_info_clean.merge(
    geo_features_clean[['ID', 'geo_features_id']],
    on='ID',
    how='left'
)

site_info_clean = site_info_clean.merge(
    climate_features_clean[['ID', 'climate_id']],
    on='ID',
    how='left'
)

site_info_clean = site_info_clean.merge(
    topo_features_clean[['ID', 'topo_features_id']],
    on='ID',
    how='left'
)

# --- Step 3: Drop 'ID' column from all involved tables ---
for table in ['site_info_clean', 'geo_features_clean', 'climate_features_clean', 'topo_features_clean']:
    df = globals().get(table)
    if df is not None and 'ID' in df.columns:
        df = df.drop(columns=['ID'])
        globals()[table] = df
        print(f"✅ Dropped 'ID' from {table}")
    else:
        print(f"⏭️ Skipping {table}: not found or no 'ID' column")

# --- District: Skip merging district_id, use district name directly ---

# don't see need for district_id, just use district name
# For district (assuming district columns are already strings)
# site_info_clean = site_info_clean.merge(
#     district_clean[['district', 'district_id']],
#     on='district',
#     how='left'
# )


✅ Dropped 'ID' from site_info_clean
✅ Dropped 'ID' from geo_features_clean
✅ Dropped 'ID' from climate_features_clean
✅ Dropped 'ID' from topo_features_clean


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_8750/2925909182.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_features_clean['ID'] = climate_features_clean['ID'].astype(str)


In [45]:
site_info_clean.head()

,site_info_id,profile,X_coord,Y_coord,district,geo_features_id,climate_id,topo_features_id
0,2770,1/57,12.161278,-15.222598,Namibe,1,1,1
1,48,1/59,12.575775,-4.866986,Cabinda,2,2,2
2,1618,1/61,15.098840,-11.225411,Cuanza Sul,3,3,3
3,881,1/63,17.081955,-9.274587,Malanje,4,4,4
4,1750,1/64,20.788116,-11.568683,Moxico,5,5,5


In [46]:
## need to drop ID from climate, geo_features, topo_features, and site_info_clean

In [47]:
site_info_clean.head()

,site_info_id,profile,X_coord,Y_coord,district,geo_features_id,climate_id,topo_features_id
0,2770,1/57,12.161278,-15.222598,Namibe,1,1,1
1,48,1/59,12.575775,-4.866986,Cabinda,2,2,2
2,1618,1/61,15.098840,-11.225411,Cuanza Sul,3,3,3
3,881,1/63,17.081955,-9.274587,Malanje,4,4,4
4,1750,1/64,20.788116,-11.568683,Moxico,5,5,5


In [48]:
profile_record_clean.head()

,profile_record_id,profile,site_info_id,soil_type_id,sample_id
0,1,139,<NA>,NaN,630
1,2,139,<NA>,NaN,631
2,3,139,<NA>,NaN,632
3,4,139,<NA>,NaN,633
4,5,139,<NA>,NaN,1817


In [49]:
## changing column headers to be SQL friendly
# Rename problematic columns
merged1 = merged1.rename(columns={
    'atm_1/3': 'atm_1_3',
    'Ca++': 'Ca',
    'Mg++': 'Mg',
    'Na+': 'Na',
    'K+': 'K',
    'Min_<0,002': 'Min_lt_0002',
    'Min_0,05-0,02': 'Min_005_002',
    'Min_0,2-0,05': 'Min_02_005',
    'Min_2-0,2': 'Min_2_02',
    'As': 'arsenic',
    'P': 'phosphorus',
    'S': 'sulfur',
    'V': 'vanadium'
})


# SAVING CLEANED TABLES TO CSV FOR DB

In [50]:
import pandas as pd

# Prepare dictionary of clean tables
tables = {
    "samples": samples_clean1,
    "analyses": merged1,
    "profile_record": profile_record_clean,
    "morphology_horizon": morphology_clean1,
    "soil_type": soil_type_clean,
    "site_info": site_info_clean,
    "climate_feat": climate_features_clean,
    "topo_feat": topo_features_clean,
    "geo_feat": geo_features_clean,
    #"districts": district_clean
}

# # Save each table to CSV with 'NULL' in empty cells
for name, df in tables.items():
#     df.replace("", pd.NA, inplace=True)  # Replace empty strings with NA
    df.to_csv(f"/Users/inesschwartz/GreenDataScience/Thesis/tables_clean/{name}.csv", index=False)

In [51]:
# Save random 100 rows of each table as mini versions for DB test
import os

# Create folder if it doesn't exist
mini_path = "/Users/inesschwartz/GreenDataScience/Thesis/tables_clean_mini"
os.makedirs(mini_path, exist_ok=True)

# Save random 100 rows of each table as mini versions
samples_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/samples.csv", index=False)
merged1.sample(n=100, random_state=42).to_csv(f"{mini_path}/analyses.csv", index=False)
profile_record_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/profile_record.csv", index=False)
morphology_clean1.sample(n=100, random_state=42).to_csv(f"{mini_path}/morphology_horizon.csv", index=False)
soil_type_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/soil_type.csv", index=False)
site_info_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/site_info.csv", index=False)
climate_features_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/climate_feat.csv", index=False)
topo_features_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/topo_feat.csv", index=False)
geo_features_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/geo_feat.csv", index=False)
#district_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/districts.csv", index=False)



In [52]:
# List your DataFrame variables here
dataframes = {
    "samples": samples_clean1,
    "analyses": merged1,
    "profile_record": profile_record_clean,
    "morphology_horizon": morphology_clean1,
    "soil_type": soil_type_clean,
    "site_info": site_info_clean,
    "climate_feat": climate_features_clean,
    "topo_feat": topo_features_clean,
    "geo_feat": geo_features_clean
    #"districts": district_clean
}

# Print column names and their datatypes
for name, df in dataframes.items():
    print(f"\n{name} ({len(df)} rows):")
    for col in df.columns:
        dtype = df[col].dtype
        print(f"  - {col} ({dtype})")



samples (14766 rows):
  - sample_id (object)
  - site_info_id (object)
  - profile (object)
  - horizon_id (object)
  - year (float64)
  - shelf (string)
  - room (object)

analyses (7847 rows):
  - lab_sample_id (object)
  - sample_id (object)
  - EG (float64)
  - thick_clay (float64)
  - fine_clay (float64)
  - silt (float64)
  - clay (float64)
  - Eq_Hum (float64)
  - atm_1_3 (float64)
  - atm_15 (float64)
  - CACO3 (float64)
  - gypsum (float64)
  - free_iron (float64)
  - organic_carbon (float64)
  - total_N (float64)
  - P205 (float64)
  - organic_material (float64)
  - pH_H2O (float64)
  - pH_KCL (float64)
  - Ca (float64)
  - Mg (float64)
  - Na (float64)
  - K (float64)
  - exchangable_bases_sum (float64)
  - CEC (float64)
  - vanadium (object)
  - conductivity (float64)
  - soluble_sodium (float64)
  - Min_lt_0002 (object)
  - Min_005_002 (object)
  - Min_02_005 (object)
  - Min_2_02 (object)
  - field_sample_code (object)
  - Depth (object)
  - Al (float64)
  - Si (float64)

In [53]:
missing = profile_record_clean[~profile_record_clean['soil_type_id'].isin(soil_type_clean['soil_type_id'])]
print(missing)

      profile_record_id profile site_info_id  soil_type_id sample_id
0                     1     139         <NA>           NaN       630
1                     2     139         <NA>           NaN       631
2                     3     139         <NA>           NaN       632
3                     4     139         <NA>           NaN       633
4                     5     139         <NA>           NaN      1817
...                 ...     ...          ...           ...       ...
86660             86661  540/67         3627           NaN     18867
86661             86662  540/67         3627           NaN     18868
86662             86663  540/67         3627           NaN     18869
86663             86664  540/67         3627           NaN     18870
86664             86665  538/67         3611           NaN     18871

[51532 rows x 5 columns]


In [54]:
samples_clean1.head()

,sample_id,site_info_id,profile,horizon_id,year,shelf,room
0,630,172,139,Hb_139/46_1_1,1946.0,1,22
1,631,173,139,Hb_139/46_2_1,1946.0,1,22
2,632,174,139,Hb_139/46_3_1,1946.0,1,22
3,633,175,139,NaN,1946.0,1,22
4,687,1034,208,Hb_208/46_1_1,1946.0,1,22


In [55]:
missing1 = site_info_clean[~site_info_clean['site_info_id'].isin(samples_clean['site_info_id'])]
print(missing1)


     site_info_id profile    X_coord    Y_coord    district  geo_features_id  \
0            2770    1/57  12.161278 -15.222598      Namibe                1   
2            1618    1/61  15.098840 -11.225411  Cuanza Sul                3   
4            1750    1/64  20.788116 -11.568683      Moxico                5   
5            3097    1/66  17.666766 -14.655526         NaN                6   
8            2675   10/54  14.445188 -14.922688       Huila                9   
...           ...     ...        ...        ...         ...              ...   
4313         1689   99/63  18.164654 -11.382465     Malanje             4314   
4314         3247   99/66  18.994294 -15.814591         NaN             4315   
4317         1485  99c/63  17.541534 -10.890781     Malanje             4318   
4319         1213   9c/63  16.357859  -9.986349     Malanje             4320   
4320         1505   9c/65  20.937136 -10.942842   Lunda Sul             4321   

      climate_id  topo_features_id  
0 

In [56]:
samples_clean1['site_info_id'].value_counts()


site_info_id
16       16
14       15
9        15
42       15
40       15
         ..
32/57     1
31/57     1
30/57     1
29/57     1
657 c     1
Name: count, Length: 6439, dtype: int64

In [57]:
samples_clean1[samples_clean1['site_info_id'].isnull()]


,sample_id,site_info_id,profile,horizon_id,year,shelf,room
10641,14219,<NA>,318/63,NaN,1963.0,458,26
12742,16639,<NA>,nan,NaN,1965.0,582,26
12743,16640,<NA>,nan,NaN,1965.0,582,26
12744,16641,<NA>,nan,NaN,1965.0,582,26
12745,16642,<NA>,nan,NaN,1965.0,582,26
12746,16643,<NA>,nan,NaN,1965.0,582,26
12747,16644,<NA>,nan,NaN,1965.0,582,26
12748,16645,<NA>,nan,NaN,1965.0,582,26
12749,16646,<NA>,nan,NaN,1965.0,582,26
12750,16647,<NA>,nan,NaN,1965.0,582,26


In [58]:
profile_record_clean.head()

,profile_record_id,profile,site_info_id,soil_type_id,sample_id
0,1,139,<NA>,NaN,630
1,2,139,<NA>,NaN,631
2,3,139,<NA>,NaN,632
3,4,139,<NA>,NaN,633
4,5,139,<NA>,NaN,1817


In [59]:
soil_type_clean.head()

,soil_type_id,profile,CEP_GR,CEP_NAME,FAO
0,1,1/51,NaN,NaN,NaN
1,2,1/57,Aridicos,Aridicos com calcario Pardo-cinzentos,CLha
2,3,1/59,Psamoferralicos,"Psamo-ferralicos Amarelos ou Alaranjados, sedi...",FRxa
3,4,1/63,Ferraliticos,Fracamente Ferralicos Vermelhos Clino-argilico...,FRh
4,5,10/54,Ferraliticos,Fracamente Ferralicos pardo-amarelados,FRh


In [60]:
# Get list of valid soil_type_ids
valid_soil_type_ids = set(soil_type_clean['soil_type_id'])

# Find invalid soil_type_ids in profile_df
invalid_rows = profile_record_clean[~profile_record_clean['soil_type_id'].isin(valid_soil_type_ids)]

# Show the result
if not invalid_rows.empty:
    print("❌ These rows reference soil_type_id values that do NOT exist in soil_type.csv:")
    print(invalid_rows)
else:
    print("✅ All soil_type_id values in profile_record.csv are valid.")


❌ These rows reference soil_type_id values that do NOT exist in soil_type.csv:
      profile_record_id profile site_info_id  soil_type_id sample_id
0                     1     139         <NA>           NaN       630
1                     2     139         <NA>           NaN       631
2                     3     139         <NA>           NaN       632
3                     4     139         <NA>           NaN       633
4                     5     139         <NA>           NaN      1817
...                 ...     ...          ...           ...       ...
86660             86661  540/67         3627           NaN     18867
86661             86662  540/67         3627           NaN     18868
86662             86663  540/67         3627           NaN     18869
86663             86664  540/67         3627           NaN     18870
86664             86665  538/67         3611           NaN     18871

[51532 rows x 5 columns]


In [61]:
profile_record_clean.head()

,profile_record_id,profile,site_info_id,soil_type_id,sample_id
0,1,139,<NA>,NaN,630
1,2,139,<NA>,NaN,631
2,3,139,<NA>,NaN,632
3,4,139,<NA>,NaN,633
4,5,139,<NA>,NaN,1817


In [62]:
# Find rows where soil_type_id is null in profile_record_clean
null_rows = profile_record_clean[profile_record_clean['site_info_id'].isnull()]

# Show the result
if not null_rows.empty:
    print("❌ These rows have NULL values in the site_info_id column of profile_record.csv:")
    print(null_rows)
else:
    print("✅ No NULL values found in the site_info_id column of profile_record.csv.")


❌ These rows have NULL values in the site_info_id column of profile_record.csv:
      profile_record_id profile site_info_id  soil_type_id sample_id
0                     1     139         <NA>           NaN       630
1                     2     139         <NA>           NaN       631
2                     3     139         <NA>           NaN       632
3                     4     139         <NA>           NaN       633
4                     5     139         <NA>           NaN      1817
...                 ...     ...          ...           ...       ...
86240             86241  508/71         <NA>           NaN     18765
86267             86268  516/68         <NA>           NaN     18772
86268             86269  516/69         <NA>           NaN     18773
86270             86271  493/68         <NA>           NaN     18781
86271             86272  493/69         <NA>           NaN     18782

[29884 rows x 5 columns]


In [63]:
# Find rows where soil_type_id is null in profile_record_clean
null_rows = profile_loc[profile_loc['ID'].isnull()]

# Show the result
if not null_rows.empty:
    print("❌ These rows have NULL values in the ID column of perfis_local.csv:")
    print(null_rows)
else:
    print("✅ No NULL values found in the ID column of perfis_local.csv.")


✅ No NULL values found in the ID column of perfis_local.csv.


In [64]:
soil_type_ids = set(soil_type_clean['soil_type_id'].unique())
profile_soil_type_ids = set(profile_record_clean['soil_type_id'].unique())

missing_ids = profile_soil_type_ids - soil_type_ids

print("soil_type_ids in profile_record missing from soil_type.csv:", missing_ids)


soil_type_ids in profile_record missing from soil_type.csv: {nan}


In [65]:
# Check if 4835 is in soil_type_id column of soil_type.csv
soil_type_has_4835 = 4835 in soil_type_clean['soil_type_id'].values

# Check if 4835 is in soil_type_id column of profile_record.csv
profile_record_has_4835 = 4835 in profile_record_clean['soil_type_id'].values

print(f"4835 in soil_type.csv? {soil_type_has_4835}")
print(f"4835 in profile_record.csv? {profile_record_has_4835}")

4835 in soil_type.csv? False
4835 in profile_record.csv? False


In [66]:
print(4835 in profile_record_clean['soil_type_id'].values)

False


In [67]:
print(4835 in profile_record_clean['soil_type_id'].values)
print(profile_record_clean[profile_record_clean['soil_type_id'] == 4835])

False
Empty DataFrame
Columns: [profile_record_id, profile, site_info_id, soil_type_id, sample_id]
Index: []


In [68]:
soil_type_ids_in_soil_type = set(soil_type_clean['soil_type_id'].unique())
soil_type_ids_in_profile_record = set(profile_record_clean['soil_type_id'].unique())

missing_soil_type_ids = soil_type_ids_in_profile_record - soil_type_ids_in_soil_type

print(f"Missing soil_type_ids in soil_type.csv: {missing_soil_type_ids}")


Missing soil_type_ids in soil_type.csv: {nan}
